# Progetto di Machine Learning

Analisi del Dataset **Dry Bean** (disponibile presso il sito [UC Irvine Machine Learning Repository](https://archive.ics.uci.edu/dataset/602/dry+bean+dataset)).

Il Dataset **Dry Bean** è stato costruito a partire da immagini ad alta risoluzione di semi di fagioli secchi appartenenti a 7 specie differenti; grazie a queste immagini è stato possibile estrarre 16 caratteristiche sui semi.

Nella cella sottostante si procede al download del dataset.

In [ ]:
# Tramite la libreria ufficiale di UCI si importa la funzione "fetch_ucirepo()" 
# utile per il download del dataset.
from ucimlrepo import fetch_ucirepo 
  
# Download del dataset tramite il codice identificativo (602)
dry_bean = fetch_ucirepo(id=602) 
  
# Si suddivide il dataset in features (X) e labels (y)
X = dry_bean.data.features 
y = dry_bean.data.targets 
y = y.to_numpy().flatten()

### Visualizzazione dei dati

Dopo aver suddivido il dataset in *features* (**X**) e *labels* (**y**) si procede con la visualizzazione dei dati contenuti all'interno del dataset **Dry Bean**

In [ ]:
# Informazioni generali sul Dataset
print(f"\nNumero di campioni -> {X.shape[0]}")
print(f"\nNumero di features -> {X.shape[1]}")

print("\nUlteriori informazioni sul dataset:")
    
# Si escludono le colonne contenenti informazioni necessarie o vuote
dry_bean.variables.drop(columns=["description", "demographic", "units"])

Si noti che all'interno del Dataset sono presenti **13611 campioni**, ognuno con **16 features** (esclusa la classe di appartenenza).

Nessuna features presenta **valori mancanti**, sono presenti solo **valori interi** (*Area* e *ConvexArea*) e **valori decimali** (tutte le restanti features).


All'interno del Dataset **Dry Bean** scaricato tramite la libreria *ucimlrepo* è presente una descrizione per ogni features.

Verrà mostrata a seguire:

In [ ]:
print("\nDescrizione delle features del dataset (Inclusa la classe):")

for i in range(len(dry_bean.variables)):
    print(f"{dry_bean.variables['name'][i]:<15} -> {dry_bean.variables['description'][i]}")

## Analisi dei dati 

Conteggio delle occorrenze per ogni classe e successiva visualizzazione grafica.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Si stampa l'intestazione per il conteggio di campioni per classe
print(f"\nNumero di campioni per classe:")

# Si ottengono i valori univoci (beans) e le loro frequenze (counts) dal vettore delle etichette y
# np.unique restituisce i valori unici e, con return_counts=True, anche le loro occorrenze
beans, counts = np.unique(y, return_counts=True)

# Si ordinano i beans in base al numero di occorrenze (dal più piccolo al più grande)
sort = np.argsort(counts)  # Si ottengono gli indici degli elementi ordinati
beans = beans[sort]        # Si riordina l'array dei beans usando questi indici
counts = counts[sort]      # Si riordina l'array dei conteggi usando gli stessi indici

# Si stampa il numero di occorrenze per ogni classe di fagioli
# Il formato {bean:<10} allinea a sinistra con una larghezza di 10 caratteri
for bean, count in zip(beans, counts):
   print(f"{bean:<10} -> {count} occorrenze")

# Si crea una figura con dimensioni specificate
plt.figure(figsize=(10, 10))

# Si crea un grafico a barre dove:
# - l'asse x rappresenta le diverse classi di fagioli (beans)
# - l'asse y rappresenta il numero di occorrenze (counts)
# - le barre sono colorate di grigio con bordo nero
plt.bar(beans, counts, color="gray", edgecolor='black')

# Si aggiungono etichette testuali sopra ogni barra con il valore esatto delle occorrenze
# count + 50 posiziona il testo 50 unità sopra la barra
# ha="center" allinea orizzontalmente il testo al centro della barra
for bean, count in zip(beans, counts):
   plt.text(bean, count + 50, f"{count}", ha="center") 
   
# Si aggiungono etichette agli assi e un titolo al grafico
plt.xlabel("Beans")  # Etichetta per l'asse x
plt.ylabel("Occorrenze")  # Etichetta per l'asse y
plt.title("Distribuzione delle occorrenze per classe", fontsize=16)  # Titolo del grafico

# Si ruotano le etichette sull'asse x di 45 gradi per migliorare la leggibilità
plt.xticks(beans, rotation=45)  

# Si aggiunge una griglia orizzontale (solo per l'asse y) con linee tratteggiate
plt.grid(axis="y", linestyle="--")  

# Si visualizza il grafico
plt.show()

Si può notare come siano presenti sostanziali differenze nella quantità di occorrenze per ogni classe, la classe meno rappresentata **BOMBAY**, ad esempio, è altamente inferiore rispetto la classe maggiormente rappresentata **DERMASON**.

Queste situazioni di *imbalance* e *over rapresentation* possono influire negativamente sui modelli, questo perchè nel caso di *imbalance* il modello potrebbe non imparare a riconoscere correttamente la classe, mentre nella situazione opposta di *over rapresentation* il modello potrebbe imparare troppo dettagliatamente a riconoscere correttamente la classe, portando ad un **OVERFITTING** o ad un **Bias** nei suoi confronti.

Tra le possibili soluzioni da considerare esistono l'**OVERSAMPLING** (ovvero la generazione sintetica di dati della classe minoritaria per bilanciare la differenza) e l'**UNDERSAMPLING** (ovvero la rimozione di dati dalla classe maggioritaria per bilanciare la differenza).

### Boxplot

Si prosegue con la visualizzazione dei dati alla ricerca di valori anomali (*outliers*) tramite **BOXPLOT**.

Per ogni Classe *BARBUNYA, BOMBAY, CALI, DERMASON, HOROZ, SEKER, SIRA* viene analizzato ogni attributo presente nel dataset **X** (features).

In [ ]:

import os

# Si crea la directory per i boxplot se non esiste
dir = "1 - BOXPLOT"
path_dir = os.path.join(os.getcwd(), dir)

if not os.path.exists(path_dir):
    os.makedirs(path_dir)

# Si crea la matrice per tenere traccia degli outlier: righe=feature, colonne=classi
# outliers[i][j] contiene il numero di outlier per la feature i nella classe j
outliers = np.zeros((len(X.columns), len(beans) + 1), dtype=int)

# Si dividono le feature in due gruppi per creare due PNG per ogni classe
features_group1 = X.columns[:8]  # Prime 8 feature
features_group2 = X.columns[8:]  # Restanti 8 feature

# Si itera per ogni classe di fagioli j (con l'aggiunta di un ciclo per l'aggiunta dei boxplot generali)
for j in range(len(beans) + 1):
    
    # GRUPPO 1 - Prime 8 feature: si crea una figura con 2 righe e 4 colonne di subplot
    fig1, axs1 = plt.subplots(2, 4, figsize=(20, 10))
    
    # Si inserisce un titolo a seconda della situazione
    if j == len(beans):
        fig1.suptitle(f"Boxplot - General 1", fontsize=25)
    else:
        fig1.suptitle(f"Boxplot - Class: {beans[j]} 1", fontsize=25)
    axs1 = axs1.flatten()  # Si trasforma la matrice 2x4 in un array 1D per facilità d'uso
    
    # GRUPPO 2 - Restanti 8 feature: si crea una seconda figura con 2 righe e 4 colonne
    fig2, axs2 = plt.subplots(2, 4, figsize=(20, 10))
    
    # Si inserisce un titolo a seconda della situazione
    if j == len(beans):
        fig2.suptitle(f"Boxplot - General 2", fontsize=25)
    else:
        fig2.suptitle(f"Boxplot - Class: {beans[j]} 2", fontsize=25)
    axs2 = axs2.flatten()  # Si trasforma la matrice 2x4 in un array 1D
    
    # Si itera sulle prime 8 feature (GRUPPO 1)
    for i, feature in enumerate(features_group1):
        ax = axs1[i]  # Si seleziona il subplot corrente
        
        if j == len(beans):
            
            # Si selezionano i dati dell'intero dataset (Boxplot - General)
            datas = X[feature]
        else:
            
            # Si filtrano solo i dati della classe corrente (j) per la feature corrente (i)
            datas = X.iloc[(y == beans[j]), X.columns.get_loc(feature)]
        
        # Si crea il boxplot con media e mediana evidenziate
        ax.boxplot(datas, 
                   showmeans=True,  # Si mostra la media
                   meanline=True,   # Si visualizza la media come linea
                   flierprops=dict(marker="x", markeredgecolor="purple"),  # Formattazione outlier
                   meanprops=dict(color="red", linestyle="--"),            # Formattazione linea media
                   medianprops=dict(color="blue", linestyle="--"))         # Formattazione linea mediana
        
        ax.set_title(f"Feature: {feature}")  # Titolo del subplot
        
        # Si calcolano gli outlier usando l'approccio del range interquartile (IQR)
        Q1 = datas.quantile(0.25)  # Primo quartile
        Q3 = datas.quantile(0.75)  # Terzo quartile
        IQR = Q3 - Q1              # Range interquartile
        
        # Si identificano gli outlier come valori fuori da [Q1-1.5*IQR, Q3+1.5*IQR]
        low_outliers = datas[datas < Q1 - 1.5 * IQR]           # Outlier inferiori
        high_outliers = datas[datas > Q3 + 1.5 * IQR]          # Outlier superiori
        tot_outliers = len(low_outliers) + len(high_outliers)  # Totale outlier
        
        # Si memorizza il numero di outlier nella matrice
        feature_idx = X.columns.get_loc(feature)
        if j == len(beans):
            
            # Si memorizzano gli outlier generali nell'ultima colonna
            outliers[feature_idx][-1] = tot_outliers
        else:
            
            # Si memorizzano gli outlier per classe
            outliers[feature_idx][j] = tot_outliers
    
    # Si itera sulle restanti 8 feature (GRUPPO 2) - procedimento identico al precedente
    for i, feature in enumerate(features_group2):
        ax = axs2[i]
        
        if j == len(beans):
            datas = X[feature]
        else:
            datas = X.iloc[(y == beans[j]), X.columns.get_loc(feature)]
        
        ax.boxplot(datas, 
                   showmeans=True, 
                   meanline=True,
                  flierprops=dict(marker="x", markeredgecolor="purple"),
                  meanprops=dict(color="red", linestyle="--"),
                  medianprops=dict(color="blue", linestyle="--")) 
       
        ax.set_title(f"Feature: {feature}")
       
        Q1 = datas.quantile(0.25)
        Q3 = datas.quantile(0.75)
        IQR = Q3 - Q1

        low_outliers = datas[datas < Q1 - 1.5 * IQR]
        high_outliers = datas[datas > Q3 + 1.5 * IQR]
        tot_outliers = len(low_outliers) + len(high_outliers)

        feature_idx = X.columns.get_loc(feature)
        if j == len(beans):
            outliers[feature_idx][-1] = tot_outliers
        else:
            outliers[feature_idx][j] = tot_outliers
   
    # Si aggiunge legenda al primo grafico (GRUPPO 1) 
    handles1 = [
        plt.Line2D([0], [0], color="red", linestyle="--", label="Media"),
        plt.Line2D([0], [0], color="blue", linestyle="--", label="Mediana"),
        plt.Line2D([0], [0], marker="x", color="purple", linestyle="", label="Outliers (x)", markersize=10)
    ]
    fig1.legend(handles=handles1, loc='upper right', ncol=3, fontsize=15)
    
    # Si aggiunge legenda al secondo grafico (GRUPPO 2)
    handles2 = [
        plt.Line2D([0], [0], color="red", linestyle="--", label="Media"),
        plt.Line2D([0], [0], color="blue", linestyle="--", label="Mediana"),
        plt.Line2D([0], [0], marker="x", color="purple", linestyle="", label="Outliers (x)", markersize=10)
    ]
    fig2.legend(handles=handles2, loc="upper right", ncol=3, fontsize=15)
    
    # Si salvano i grafici in file separati
    plt.figure(fig1.number)
    if j == len(beans):
        plt.savefig(f"{dir}/ALL_features_1_8")
    else:
        plt.savefig(f"{dir}/{beans[j]}_features_1_8")
    
    plt.figure(fig2.number)
    if j == len(beans):
        plt.savefig(f"{dir}/ALL_features_9_16")
    else:
        plt.savefig(f"{dir}/{beans[j]}_features_9_16")
    
    # Si chiudono le figure per liberare memoria
    plt.close(fig1)
    plt.close(fig2)

**Spiegazione Boxplot**

La base inferiore della scatola rappresenta il **primo quartile** ($Q1$), ovvero il valore che separa il 25% inferiore dei dati; mentre la base superiore rappresenta il **terzo quartile** ($Q3$), ovvero il valore che separa il 75% superiore dei dati.

La lunghezza della scatola rappresenta l'**intervallo interquartile** ($IQR$), ovvero la differenza tra il **terzo quartile** ($Q3$) e il **primo quartile** ($Q1$), al suo interno sono presenti la metà centrale dei dati. 

Inoltre sono visibili la linea **mediana** (nei file .png rappresentanti i Boxplot è visualizzata come una linea di colore blu) che rappresenta il **secondo interquartile** ($Q2$) e la linea della **media** (nei file .png rappresentanti i Boxplot è visualizzata come una linea di colore rosso) che rappresenta il valore medio di tutti i dati.

I baffi, invece, sono linee che si estendono dall'estremità della scatola sino ai dati che non sono considerati outliers.

Gli **outliers**, infine, sono dati (*valori anomali*) che si trovano al di fuori dei baffi e non rispettano le formule $Q1 - 1.5 * IQR$ e $Q3 + 1.5 * IQR$(le quali determinano quali sono i valori anomali).

**Valutazione dei Boxplot ottenuti**

Visualizzando i vari file .png si può notare come le distribuzioni dei dati presentano quasi sempre outliers, di seguito si verifica il totale di outliers ottenuti rispetto ogni features, e la sua percentuale corrispondente.

In [ ]:
import pandas as pd

print("Numero di Outliers per ogni classe e feature:")

# Si crea un DataFrame con le colonne per ogni classe, più una colonna "General"
columns_names = list(beans) + ["General"]
outliers_df = pd.DataFrame(
    data=outliers, 
    index=X.columns,       
    columns=columns_names
)

# Si sommano gli outlier escludendo la colonna General
outliers_df["TOT (escluso General)"] = outliers_df.iloc[:, :-1].sum(axis=1)  

# Stampa del DataFrame
print(outliers_df.to_string())

print("\nPercentuale corrispondente:")
beans_counts = dict(zip(beans, counts))

# Si aggiunge il totale dei campioni per il calcolo della percentuale generale
beans_counts_with_general = beans_counts.copy()
beans_counts_with_general["General"] = sum(counts)

# Si calcolano le percentuali
outliers_df_perc = outliers_df.iloc[:, :-1].div(pd.Series(beans_counts_with_general), axis=1) * 100

totale_campioni = sum(counts)
outliers_df_perc["TOT (escluso General)"] = (outliers_df["TOT (escluso General)"] / totale_campioni * 100).round(2)

outliers_df_perc = outliers_df_perc.round(2)

# Stampa del DataFrame
print(outliers_df_perc.to_string())


Si può verificare che per ciacuna classe, nessuna feature presenta una percentuale maggiore del 10% di **outliers**, mentre sul totale non viene superato il 5% (escludendo la colonna General). Può rendersi comunque necessaria la gestione dei valori anomali. 

### Istogramma

Per sicurezza si verifica la distribuzione dei dati in un istogramma.

In [ ]:
# Si crea la directory per gli istogrammi se non esiste
dir = "2 - HISTOGRAM"
path_dir = os.path.join(os.getcwd(), dir)

if not os.path.exists(path_dir):
   os.makedirs(path_dir)
   
# Si dividono le feature in due gruppi per creare due PNG per ogni classe
features_group1 = X.columns[:8]  # Prime 8 feature
features_group2 = X.columns[8:]  # Restanti 8 feature

# Si itera per ogni classe di fagioli j (con l'aggiunta di un ciclo per l'aggiunta degli istogrammi generali)
for j in range(len(beans) + 1):
   
   # GRUPPO 1 - Prime 8 feature: si crea una figura con 2 righe e 4 colonne di subplot
   fig1, axs1 = plt.subplots(2, 4, figsize=(20, 10))
   
   # Si inserisce un titolo a seconda della situazione
   if j == len(beans):
      fig1.suptitle(f"Histogram - General 1", fontsize=25)
   else:
      fig1.suptitle(f"Histogram - Class: {beans[j]} 1", fontsize=25)
   axs1 = axs1.flatten()  # Si trasforma la matrice 2x4 in un array 1D per facilità d'uso
   
   # GRUPPO 2 - Restanti 8 feature: si crea una seconda figura con 2 righe e 4 colonne
   fig2, axs2 = plt.subplots(2, 4, figsize=(20, 10))
   
   # Si inserisce un titolo a seconda della situazione
   if j == len(beans):
      fig2.suptitle(f"Histogram - General 2", fontsize=25)
   else:
      fig2.suptitle(f"Histogram - Class: {beans[j]} 2", fontsize=25)
   axs2 = axs2.flatten()  # Si trasforma la matrice 2x4 in un array 1D

   # Si itera sulle prime 8 feature (GRUPPO 1)
   for i, feature in enumerate(features_group1):
      ax = axs1[i]  # Si seleziona il subplot corrente
      
      if j == len(beans):
         
         # Si selezionano i dati dell'intero dataset
         datas = X[feature]
      else:
         
         # Si filtrano solo i dati della classe corrente (j) per la feature corrente (i)
         datas = X.iloc[(y == beans[j]), X.columns.get_loc(feature)]
       
      # Si calcolano media e deviazione standard
      mean = np.mean(datas)
      std = np.std(datas)
       
      # Si crea l'istogramma con media e deviazione standard evidenziate
      ax.hist(datas, bins="auto", edgecolor="black", color="purple", alpha=0.5)
      ax.axvline(mean, color="red", linestyle="--")  # Si visualizza la media
      ax.axvline(mean - std, color="blue", linestyle="dashed")  # Si visualizza media - deviazione standard
      ax.axvline(mean + std, color="blue", linestyle="dashed")  # Si visualizza media + deviazione standard
      
      # Si imposta il titolo del subplot con i valori statistici
      if j == len(beans):
         ax.set_title(f"Feature: {feature}\nμ = {mean:.2f} - σ = {std:.2f}") 
      else:
         ax.set_title(f"Feature: {feature}\nClass {beans[j]}: μ = {mean:.2f} - σ = {std:.2f}") 

   # Si itera sulle restanti 8 feature (GRUPPO 2) - procedimento identico al precedente
   for i, feature in enumerate(features_group2):
      ax = axs2[i] 
      
      if j == len(beans):
         datas = X[feature]
      else:
         datas = X.iloc[(y == beans[j]), X.columns.get_loc(feature)]
      
      mean = np.mean(datas)
      std = np.std(datas)
      
      ax.hist(datas, bins="auto", edgecolor="black", color="purple", alpha=0.5)
      ax.axvline(mean, color="red", linestyle="--")  
      ax.axvline(mean - std, color="blue", linestyle="dashed")  
      ax.axvline(mean + std, color="blue", linestyle="dashed")  
      
      if j == len(beans):
         ax.set_title(f"Feature: {feature}\nμ = {mean:.2f} - σ = {std:.2f}") 
      else:
         ax.set_title(f"Feature: {feature}\nClass {beans[j]}: μ = {mean:.2f} - σ = {std:.2f}") 
   
   # Si aggiunge legenda al primo grafico (GRUPPO 1) 
   handles1 = [
      plt.Line2D([0], [0], color="red", linestyle="--", label="Media (μ)"),
      plt.Line2D([0], [0], color="blue", linestyle="--", label="Deviazione Standard (σ)"),
   ]
   fig1.legend(handles=handles1, loc='upper right', ncol=2, fontsize=15)

   # Si aggiunge legenda al secondo grafico (GRUPPO 2)
   handles2 = [
      plt.Line2D([0], [0], color="red", linestyle="--", label="Media (μ)"),
      plt.Line2D([0], [0], color="blue", linestyle="--", label="Deviazione Standard (σ)"),
   ]
   fig2.legend(handles=handles2, loc='upper right', ncol=2, fontsize=15)
   
   # Si salvano i grafici in file separati
   plt.figure(fig1.number)
   if j == len(beans):
      plt.savefig(f"{dir}/ALL_features_1_8")
   else:
      plt.savefig(f"{dir}/{beans[j]}_features_1_8")
   
   plt.figure(fig2.number)
   if j == len(beans):
      plt.savefig(f"{dir}/ALL_features_9_16")
   else:
      plt.savefig(f"{dir}/{beans[j]}_features_9_16")
   
   # Si chiudono le figure per liberare memoria
   plt.close(fig1)
   plt.close(fig2)

**Spiegazione Istogramma**

L'istogramma è una rappresentazione grafica della *distribuzione di frequenza dei dati* che permette di visualizzarne la forma complessiva. Ogni barra rappresenta la frequenza dei valori all'interno di un determinato intervallo.

Negli istogrammi generati, la linea verticale rossa rappresenta la **media** (μ) della distribuzione, mentre le linee blu tratteggiate indicano la **deviazione standard** (σ), mostrando l'intervallo μ ± σ. Questo intervallo, in una distribuzione normale, contiene circa il 68% dei dati.

L'istogramma fornisce informazioni importanti come la *tendenza centrale dei dati*, la *forma della distribuzione*, la *dispersione dei dati* e la *presenza di valori anomali* o di raggruppamenti inaspettati

Attraverso l'analisi degli istogrammi per ciascuna classe e feature, si possono identificare caratteristiche distintive delle diverse varietà di fagioli e comprendere la variabilità all'interno di ciascuna classe.

**Valutazione degli Istogrammi ottenuti**

In linea generale, la maggior parte delle distribuzioni segue una forma a campana, suggerendo una **distribuzione normale** (o simil normale), alcune classi presentano una distribuzione più ampia (maggiore deviazione standard).
Le distribuzioni Generali invece presentano distribuzioni con più picchi, evidenziando una differenza nelle distribuzioni delle features.

In [ ]:
import seaborn as sns

# Si crea la directory per le matrici di correlazione se non esiste
dir = "3 - MATRIX CORRELATION"
path_dir = os.path.join(os.getcwd(), dir)

if not os.path.exists(path_dir):
    os.makedirs(path_dir)

# Si itera per ogni classe di fagioli j
for j in range(len(beans)):
    
    # Si filtrano solo i dati della classe corrente (j)
    datas = X.iloc[(y == beans[j])]
    
    # Si calcola la matrice di correlazione per la classe corrente
    correlation_matrix = datas.corr()
    
    # Si crea la figura per la matrice di correlazione
    plt.figure(figsize=(15, 10))
    
    # Si visualizza la matrice come heatmap con annotazioni
    sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)
    
    # Si aggiunge il titolo alla figura
    plt.title(f"Feature Correlation Matrix - Class {beans[j]}", fontsize=25)
    
    # Si salva la figura nella directory appropriata
    plt.savefig(f"{dir}/correlation_matrix_{beans[j]}.png")
    
    # Si chiude la figura per liberare memoria
    plt.close()

# Si calcola la matrice di correlazione generale (considerando tutti i dati)
correlation_matrix_all = X.corr()

# Si crea la figura per la matrice di correlazione generale
plt.figure(figsize=(15, 10))

# Si visualizza la matrice generale come heatmap con annotazioni
sns.heatmap(correlation_matrix_all, annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)

# Si aggiunge il titolo alla figura generale
plt.title("General Correlation Matrix", fontsize=25)

# Si salva la figura generale nella directory appropriata
plt.savefig(f"{dir}/ALL_correlation_matrix.png")

# Si chiude la figura per liberare memoria
plt.close()

**Spiegazione della Matrice di Correlazione**

La matrice di correlazione è una tabella che mostra i coefficienti di correlazione tra coppie di variabili (feature). Ogni cella della matrice rappresenta il coefficiente di correlazione di Pearson tra due feature, indicando la forza e la direzione della relazione lineare tra esse.

Il coefficiente di correlazione varia da -1.0 a +1.0, dove:
- Un valore di +1.0 indica una **correlazione positiva perfetta** (quando una variabile aumenta, l'altra aumenta proporzionalmente)
- Un valore di -1.0 indica una **correlazione negativa perfetta** (quando una variabile aumenta, l'altra diminuisce proporzionalmente)
- Un valore di 0 indica l'**assenza di correlazione lineare**

Nella visualizzazione, i colori caldi (rossi) indicano **correlazioni positive**, mentre i colori freddi (blu) indicano **correlazioni negative**. L'intensità del colore rappresenta la forza della correlazione: colori più intensi indicano correlazioni più forti.

La diagonale della matrice mostra la correlazione di ciascuna variabile con se stessa, che è sempre 1.0. La matrice è simmetrica rispetto alla diagonale, poiché la correlazione tra la variabile A e B è la stessa che tra B e A.

Analizzando le matrici di correlazione per le diverse classi di fagioli, si possono identificare:
- Feature fortemente correlate che potrebbero essere ridondanti
- Relazioni specifiche tra caratteristiche morfologiche per ciascuna varietà
- Differenze nei pattern di correlazione tra le diverse classi, che potrebbero riflettere differenze biologiche fondamentali

La matrice di correlazione è quindi uno strumento essenziale per comprendere le interrelazioni tra le caratteristiche.

**Valutazione delle Matrici di Correlazione ottenute**

Verificando i risultati ottenuti, si può affermare che le varie matrici di coorelazione ottenute dalle diverse classi di fagioli hanno struttura simile, sia nelle correlazioni positive che negative. Ciò si può notare anche dall'analisi della matrice di correlazione generale.

## Preprocessing

ll data pre-processing, o pre-elaborazione dei dati, è una fase fondamentale nel processo di preparazione dei dati per l'applicazione di algoritmi di machine learning. Questa fase coinvolge una serie di azioni volte a pulire, trasformare e preparare i dati in modo da renderli adatti all'analisi e all'addestramento degli algoritmi di machine learning.

### Standardizzazione

Tecnica comune utilizzata per garantire che le varie feature abbiano la stessa scala. 

A seguito della standardizzazione si ottiene una **distribuzione delle feature** con *media* = 0 e *deviazione standard* = 1, denominata distribuzione normale standardizzata.

In [ ]:
from sklearn.preprocessing import StandardScaler

# Funzione per la standardizzazione del dataset
# Si inizializza lo StandardScaler per standardizzare i dati

scaler = StandardScaler() # Si dichiara fuori dalla funzione così da poterlo riusare 
def standardization(X_train, X_test):

    # Si applica la standardizzazione ai dati : trasforma i dati in modo che 
    # abbiano media = 0 e deviazione standard = 1
    X_train_std = scaler.fit_transform(X_train)
    X_test_std = scaler.transform(X_test)
    
    return X_train_std, X_test_std

### Normalizzazione

Tecnica comune utilizzata per scalare le feature in un intervallo specifico, tipicamente [0,1]. 

La normalizzazione mappa i valori minimi e massimi di ciascuna feature rispettivamente a 0 e 1, mantenendo la distribuzione proporzionale dei dati originali. Questo approccio è particolarmente utile quando le feature hanno range diversi.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Si inizializza il MinMaxScaler per normalizzare i dati 
normalizer = MinMaxScaler() # Si dichiara fuori dalla funzione così da poterlo riusare

def normalization(X_train, X_test):
    
    # Si applica la normalizzazione ai dati: trasforma i dati in modo che
    # siano nell'intervallo [0,1] mantenendo le proporzioni originali
    X_train_norm = normalizer.fit_transform(X_train)
    X_test_norm = normalizer.transform(X_test)
    
    return X_train_norm, X_test_norm

### Aggregazione delle feature

L'aggregazione di feature rappresenta una tecnica avanzata di **riduzione della dimensionalità** che opera attraverso il raggruppamento gerarchico di feature simili. Questo approccio permette di combinare variabili correlate in modo interpretabile, preservando la struttura informativa del dataset.

Basandoci sulla General Correlation Matrix ottenuta, si può affermare che è presente un'elevata multicollineareità tra features. La matrice evidenzia chiaramente gruppi di variabili fortemente correlate (con coefficienti superiori a 0.9), di conseguenza l'aggregazione può essere un'ottima soluzione per ridurre la dimensionalità senza perdere informazioni significative.

Per procedere correttamente con l'aggregazione **è necessario standardizzare e/o normalizzare i dati**. Questo passaggio preliminare è fondamentale per garantire che il processo di clustering non sia influenzato dalle diverse scale delle variabili originali ma si basi esclusivamente sui pattern di correlazione.

Il dendrogramma risultante dall'analisi gerarchica fornisce una rappresentazione visiva delle relazioni tra le feature e permette di identificare il numero ottimale di cluster da utilizzare per l'aggregazione finale.

In [ ]:
from scipy.cluster import hierarchy

# Si crea la directory per i dendrogrammi se non esiste
dir = "4 - DENDROGRAM"
path_dir = os.path.join(os.getcwd(), dir)

if not os.path.exists(path_dir):
    os.makedirs(path_dir)

# Si crea una figura con due subplot affiancati orizzontalmente
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))

# PRIMO SUBPLOT - Dendrogramma con dati standardizzati
# Si calcola la matrice di distanza tra feature standardizzate 
# Nota: si traspone la matrice per ottenere feature × osservazioni
distance_matrix_std = hierarchy.distance.pdist(scaler.fit_transform(X).T)

# Si applica il linkage gerarchico con metodo Ward
linkage_matrix_std = hierarchy.linkage(distance_matrix_std, method="ward")

# Si crea il dendrogramma per i dati standardizzati nel primo subplot
dendrogram_std = hierarchy.dendrogram(
    linkage_matrix_std,
    labels=X.columns,
    leaf_rotation=90,
    leaf_font_size=10,
    color_threshold=0,
    ax=ax1  # Si specifica il primo subplot
)

# Si impostano titolo e assi per il primo subplot
ax1.set_title("Dendrogram (StandardScaler)", fontsize=15)
ax1.set_xlabel("Feature")
ax1.set_ylabel("Dissimilarity")

# SECONDO SUBPLOT - Dendrogramma con dati normalizzati
# Si calcola la matrice di distanza tra feature normalizzate
distance_matrix_norm = hierarchy.distance.pdist(normalizer.fit_transform(X).T)

# Si applica il linkage gerarchico con metodo Ward
linkage_matrix_norm = hierarchy.linkage(distance_matrix_norm, method="ward")

# Si crea il dendrogramma per i dati normalizzati nel secondo subplot
dendrogram_norm = hierarchy.dendrogram(
    linkage_matrix_norm,
    labels=X.columns,
    leaf_rotation=90,
    leaf_font_size=10,
    color_threshold=0,
    ax=ax2  # Si specifica il secondo subplot
)

# Si impostano titolo e assi per il secondo subplot
ax2.set_title("Dendrogram (MinMaxScaler)", fontsize=15)
ax2.set_xlabel("Feature")
ax2.set_ylabel("Dissimilariy")

# Si aggiunge un titolo generale alla figura
fig.suptitle("Dendrogram with standardization - Dendrogram with normalization", fontsize=20)

# Si salva la figura combinata nella directory appropriata
plt.savefig(f"{dir}/Dendrogram.png")

# Si chiude la figura per liberare memoria
plt.close()

Dal file PNG ottenuto, si può verificare come a seconda del metodo di pre-processing utilizzato variano i risultati dell'aggregazione.

Si può affermare che nel caso dell'aggregazione successiva alla standardizzazione si individuano **6 cluster** differenti.

Nel caso dell'aggregazione successiva alla normalizzazione, si individuano **4 cluster** principali.

Si aggiunge alla cartella DENDROGRAM la visualizzazione della separazione per cluster.

In [ ]:
from scipy.cluster import hierarchy

# Si rimane nella directory precedentemente creata
dir = "4 - DENDROGRAM"
path_dir = os.path.join(os.getcwd(), dir)

# Si crea una figura con due subplot affiancati orizzontalmente
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))

# PRIMO SUBPLOT - Dendrogramma con dati standardizzati

# Si determina il numero di cluster e il threshold 
# threshold è il parametro che determina quali cluster evidenziare
n_clusters_std = 6
threshold_std = linkage_matrix_std[-(n_clusters_std-1), 2]

# Si crea il dendrogramma per i dati standardizzati nel primo subplot
dendrogram_std = hierarchy.dendrogram(
    linkage_matrix_std,
    labels=X.columns,
    leaf_rotation=90,
    leaf_font_size=10,
    color_threshold=threshold_std,
    ax=ax1  # Si specifica il primo subplot
)

# Si impostano titolo e assi per il primo subplot
ax1.set_title("Dendrogram (StandardScaler)", fontsize=15)
ax1.set_xlabel("Feature")
ax1.set_ylabel("Dissimilarity")

# Si determina il numero di cluster e il threshold 
# threshold è il parametro che determina quali cluster evidenziare
n_clusters_norm = 4
threshold_norm = linkage_matrix_norm[-(n_clusters_norm-1), 2]

# Si crea il dendrogramma per i dati normalizzati nel secondo subplot
dendrogram_norm = hierarchy.dendrogram(
    linkage_matrix_norm,
    labels=X.columns,
    leaf_rotation=90,
    leaf_font_size=10,
    color_threshold=threshold_norm,
    ax=ax2  # Si specifica il secondo subplot
)

# Si impostano titolo e assi per il secondo subplot
ax2.set_title("Dendrogram (MinMaxScaler)", fontsize=15)
ax2.set_xlabel("Feature")
ax2.set_ylabel("Dissimilariy")

# Si aggiunge un titolo generale alla figura
fig.suptitle("Dendrogram with standardization - Dendrogram with normalization", fontsize=20)

# Si salva la figura combinata nella directory appropriata
plt.savefig(f"{dir}/Dendrogram_colored.png")

# Si chiude la figura per liberare memoria
plt.close()
# Si stampa un messaggio informativo per i cluster ottenuti dai dati standardizzati
print("Cluster ottenuti dai dati aggregati con standardizzazione")

# Si ottengono le etichette dei cluster dal linkage matrix dei dati standardizzati
# utilizzando il numero di cluster specificato
cluster_labels = hierarchy.fcluster(linkage_matrix_std, n_clusters_std, criterion='maxclust')

# Si estraggono i valori unici delle etichette dei cluster
unique_clusters = np.unique(cluster_labels)

# Si iterano i cluster unici
for i in unique_clusters:
    
   # Si estraggono i nomi delle feature appartenenti al cluster corrente
   cluster_members = [col for col, label in zip(X.columns, cluster_labels) if label == i]
   
   # Si stampano le informazioni sul cluster
   print(f"Cluster {i}: {cluster_members}")

# Si stampa un messaggio informativo per i cluster ottenuti dai dati normalizzati
print("\nCluster ottenuti dai dati aggregati con normalizzazione")

# Si ottengono le etichette dei cluster dal linkage matrix dei dati normalizzati
# utilizzando il numero di cluster specificato
cluster_labels = hierarchy.fcluster(linkage_matrix_norm, n_clusters_norm, criterion='maxclust')

# Si estraggono i valori unici delle etichette dei cluster
unique_clusters = np.unique(cluster_labels)

# Si iterano i cluster unici
for i in unique_clusters:
    
   # Si estraggono i nomi delle feature appartenenti al cluster corrente
   cluster_members = [col for col, label in zip(X.columns, cluster_labels) if label == i]
   
   # Si stampano le informazioni sul cluster
   print(f"Cluster {i}: {cluster_members}")

Una volta creati i cluster (confrontabili con il dendrogramma visualizzato precedentemente), si può procedere all'aggregazione dei dati standardizzati o normalizzati con l'apposita funzione (*i cluster otttenuti possono variare leggermente a causa della diversa funzione utilizzata*).

In [ ]:
from sklearn.cluster import FeatureAgglomeration
from typing import Literal

def agglomeration(X_train, X_test, type: Literal["normalization", "standardization"]):
    
    # Si verifica il parametro
    if type not in ["normalization", "standardization"]:
        raise ValueError("cluster_type deve essere 'normalization' o 'standardization'")
    
    # Si imposta il numero corretto di cluster da generare a seconda della 
    # tecnica precentemente utilizzata
    if type == "normalization":
        clusters = n_clusters_norm
    elif type == "standardization":
        clusters = n_clusters_std
    
    # Si utilizza FeatureAgglomerator per ridurre la dimensionalità 
    # raggruppando feature simili basandosi su criteri di distanza.
    # Si utilizzano le stesse impostazioni utilizzate per il dendrogramma
    # così da ottenere i medesimi cluster
    agglomerator = FeatureAgglomeration(
        n_clusters=clusters,
        metric="euclidean",
        linkage="ward"
    )
    
    # Si applica l'aggregazione ai dati: Si aggregano le feature sulla base
    # delle impostazioni di FeatureAgglomeration
    X_train_agg = agglomerator.fit_transform(X_train)
    X_test_agg = agglomerator.transform(X_test)
    
    return X_train_agg, X_test_agg

### Undersampling

L'undersampling è una tecnica di riduzione dei dati che consiste nella selezione di un sottoinsieme rappresentativo di osservazioni da un dataset originale più grande. Questa strategia viene utilizzata principalmente per ridurre il costo computazionale e migliorare l'efficienza dell'analisi.

L'undersampling offre numerosi vantaggi in termini di efficienza, tuttavia è importante considerare il rischio di perdita di informazioni.

Si usa la tecnica ClusterCentroids, individuando i centroidi rappresentativi delle classi maggioritarie (tramite Kmeans) e riducendole a un numero di istanze pari a quello della classe minoritaria. Nonostante ClusterCentroids generi dati sintetici preserva meglio la distribuzione originale dei dati, cattura la struttura interna di ogni classe maggioritaria e riduce il rumore mantenendo l'informazione essenziale.

In [ ]:
from imblearn.under_sampling import ClusterCentroids

# Si inizializza ClusterCentroids 
cc = ClusterCentroids(random_state=21) # Si dichiara fuori dalla funzione così da poterlo riusare

def undersampling(X_train, y_train):
    
    # Si applica l'undersampling ai dati di train
    X_train_und, y_train_und = cc.fit_resample(X_train, y_train)
    
    return X_train_und, y_train_und



### Oversampling

L'oversampling è una tecnica di espansione dei dati che consiste nell'aumentare artificialmente il numero di istanze nelle classi minoritarie di un dataset sbilanciato. Questa strategia viene utilizzata principalmente per bilanciare la distribuzione delle classi e migliorare le prestazioni dei modelli di classificazione.

L'oversampling offre numerosi vantaggi in termini di bilanciamento del dataset, tuttavia è importante considerare il rischio di overfitting sui dati sintetici generati.

Si usa la tecnica SMOTE (Synthetic Minority Over-sampling Technique), che genera esempi sintetici per le classi minoritarie creando nuove istanze lungo i segmenti che collegano i k vicini più prossimi di ciascun esempio. A differenza della duplicazione casuale, SMOTE crea esempi che seguono la distribuzione originale dei dati, introducendo variabilità che aiuta il modello a generalizzare meglio e a definire confini decisionali più robusti.

In [ ]:
from imblearn.over_sampling import SMOTE

# Si inizializza Smote
smote = SMOTE(random_state=21) # Si dichiara fuori dalla funzione così da poterlo riusare

def oversampling(X_train, y_train):
    
    # Si applica l'oversampling sui dati di train
    X_train_und, y_train_und = smote.fit_resample(X_train, y_train)
    
    return X_train_und, y_train_und

Di seguito si genera una lista di liste contenente le varie combinazioni possibili data dalle combinazioni delle varie tecniche di preprocessing utilizzate.

Successivamente si definisce una funzione per la combianzione di quest'ultime.

In [ ]:
from sklearn.model_selection import train_test_split

# Si definisce la lista di liste techniques contenente tutte le possibili combinazioni
# delle varie tecniche di pre processing (compreso il caso in cui non si utilizzano tecniche)
techniques = [
    [],
    ["standardization"],
    ["normalization"],
    ["undersampling"],
    ["oversampling"],
    ["standardization", "undersampling"],
    ["standardization", "oversampling"],
    ["normalization", "undersampling"],
    ["normalization", "oversampling"],
    ["standardization", "agglomeration"],
    ["normalization", "agglomeration"],
    ["standardization", "agglomeration", "undersampling"],
    ["standardization", "agglomeration", "oversampling"],
    ["normalization", "agglomeration", "undersampling"],
    ["normalization", "agglomeration", "oversampling"]
]

# Si definisce una funzione per la combinazione delle varie tecniche
def combine_preprocessing(X, y, techniques):
    
    # Si utilizza lo split con stratificazione per mantenere il bilanciamento delle classe
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify=y)

    # Standardizzazione o normalizzazione (mutuamente esclusivi)
    if "standardization" in techniques:
        X_train, X_test = standardization(X_train, X_test)
    elif "normalization" in techniques:
        X_train, X_test = normalization(X_train, X_test)
    
    # Agglomerazione (DOPO standardizzazione/normalizzazione)
    if "agglomeration" in techniques:
        if "standardization" in techniques:
            X_train, X_test = agglomeration(X_train, X_test, "standardization")
        elif "normalization" in techniques:
            X_train, X_test = agglomeration(X_train, X_test, "normalization")
    
    # Undersampling o oversampling (mutuamente esclusivi)
    if "undersampling" in techniques:
        X_train, y_train = undersampling(X_train, y_train)
    elif "oversampling" in techniques:
        X_train, y_train = oversampling(X_train, y_train)
    
    return X_train, X_test, y_train, y_test

## Modelli di Classificazione

I modelli di classificazione sono **algoritmi di machine learning** che assegnano un'etichetta di classe predefinita a nuove osservazioni, basandosi su un *training set* di esempi già etichettati. Questi modelli apprendono i pattern nei dati per effettuare previsioni su nuovi esempi. L'implementazione segue generalmente quattro fasi: **preparazione dei dati**, **selezione del modello appropriato** in base alle caratteristiche del problema, **addestramento sui dati di training** per ottimizzare i parametri interni e **valutazione delle performance** su dati non visti. È essenziale valutare le prestazioni con metriche appropriate come accuratezza, precisione e recall, e ottimizzare gli iperparametri.

Si valuteranno una serie di algoritmi per verificare quale ha l'accuratezza maggiore e su quale dataset

In [ ]:
# Si crea una lista per contenere i risultati e le caratteristiche dei migliori modelli
best_configs_list = []

### Decision Tree

L'**albero decisionale** è un algoritmo di machine learning che prende decisioni seguendo una struttura ad albero, dove ogni nodo interno rappresenta un test su un attributo, ogni ramo corrisponde a un risultato del test e ogni foglia rappresenta una classe o una decisione finale. 

Gli alberi decisionali offrono notevoli vantaggi: sono facilmente interpretabili, gestiscono naturalmente dati categorici e numerici, trattano efficacemente valori mancanti e catturano relazioni non lineari senza richiedere trasformazioni preliminari dei dati. Hanno tuttavia la tendenza all'**overfitting**.

La libreria *scikit-learn* mette a disposizione la classe [*DecisionTreeClassifier*](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) per implementare l'algoritmo di classificazione dell'albero decisionale. Quest'ultimo utilizza l'indice di Gini by default per misurare la qualità degli split. 

Si effettua un **tuning degli iperparametri** per definire il valore migliore per la **profondità** dell'albero (DecisionTrees) e quale criterio è migliore per la misurazione della **qualità degli split** (gini, entropy, log_loss), si eseguono una serie di passaggi:
- Si definisce *StratifiedKFold*, un generatore di split utile per la preparazione di dati per la cross validation successiva. *StratifiedKFold* divide il dataset in 5 fold, durante la cross validation verranno eseguite 5 iterazioni, ogni iterazione userà 4 fold per il training e 1 fold per la validation.
- Si utilizza *train_test_split* per splittare il dataset iniziale (**X**, **y**) in train e test con stratificazione (si utilizza solo il train per la cross validation) 
- Si utilizza la tecnica *GridSearchCV* come tecnica di cross validation in icerca la migliore combinazione di iperparametri provando automaticamente tutte le combinazioni specificate. Inoltre ha un costo computazionale inferiore rispetto le classiche tecniche di cross validation e restituisce il miglior modello.

Per ogni combinazione di tecniche si restituisce l'accuratezza maggiore in corrispondenza della profondità ottimale dell'albero e del miglior criterio di separazione.

In [ ]:
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier

# Si definisce skf, si divide il dataset in 5 fold con shuffle
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=21)

# Si definisce una lista per contenere i risultati
results_decisionTrees = []

# Si definisce il classificatore DecisionTreeClassifier su cui effettuare
# il tuning degli iperparametri
clf = DecisionTreeClassifier(random_state=21)

# Si definisce una lista di profondità dell'albero da testare 
# durante la cross validation
max_depth = list(range(5,16)) # 5 - 15

# Si definisce una lista dei possibili criteri da utilizzare
# per misurare la qualità degli split
criterion = ['gini', 'entropy']

# Si divide il dataset iniziale in train e test con stratificazione 
# ovvero le occorrenze delle classi sono automaticamente bilanciate
# tra train e test
X_train, _, y_train, _ = train_test_split(X, y, test_size=0.2, random_state=21, stratify=y)


# Si inizializzano le due variabili utilizzate successivamente per salvare il miglior
# classificatore ttrovato tramite la grid search e cross validation
best_decisionTree_score = -1
best_decisionTree_clf = None

# Si itera per tutte le combinazioni di tecniche di pre processing possibili 
# e precedentemente definite
for technique in techniques:
    
    # Si definisce una lista per contenere i vari step per la cross validation
    steps = []
    
    # Se la tecnica specificata tra virgolette è presente nella lista
    # technique si aggiunge tra gli step della cross validation
    if "standardization" in technique:
        steps.append(("scaler", scaler))
        
        # Si salva il numero corretto di cluster per la standardizzazione 
        n_clusters = n_clusters_std
        
    elif "normalization" in technique:
        steps.append(("normalizer", normalizer))
        
        # Si salva il numero corretto di cluster per la normalizzazione
        n_clusters = n_clusters_std
        
    if "agglomeration" in technique:
        
        # Si definisce lo step dell'agglomerazione di feature con gli stessi 
        # parametri usati precedentemente 
        steps.append(("agglomeration", FeatureAgglomeration(
            n_clusters=n_clusters,
            metric="euclidean",
            linkage="ward"
        )))
        
    if "undersampling" in technique:
        steps.append(("undersampler", cc))
        
    elif "oversampling" in technique:
        steps.append(("oversampler", smote))
    
    # Si aggiunge il classificatore DecisionTreeClassifier come ultimo step
    # della cross validation
    steps.append(("classifier", clf))
    
    # Si crea la pipeline di step 
    # si utilizza ImbPipeline invece di Pipeline in quanto accetta 
    # le tecniche di sampling (oversampling/undersampling)
    pipeline = ImbPipeline(steps)

    # Si definisce il parametro di GridSearchCV per la ricerca della
    # migliore profondità dell'albero e del miglior criterio
    param_grid = {
        "classifier__max_depth": max_depth,
        "classifier__criterion": criterion
    }
    
    # Si definisce GridSearchCV, la pipeline da seguire, i parametri,
    # la tecnica per la cross validation (StratifiedKFold), lo scoring da 
    # utilizzare (balanced_accuracy in quanto il dataset è sbilanciato) e il
    # numero di core da utilizzare contemporaneamente
    grid_search = GridSearchCV(
        estimator=pipeline,
        param_grid=param_grid,
        cv=skf,
        scoring="balanced_accuracy",
        n_jobs=-1,                      # si utilizzano tutti i core disponibili
        verbose=1                       # si mostrano degli aggiornamenti sull'andamento del codice
    )

    # Si esegue la cross validation sul train set, per ogni tecnica si 
    # verifica quale combinazione di profondità e criterio ottiene l'accuratezza maggiore
    grid_search.fit(X_train, y_train)
    
    # Si verifica il risultato del modello, se migliore del precedente si salva il corrente
    if grid_search.best_score_ > best_decisionTree_score:
        best_decisionTree_score = grid_search.best_score_
        best_decisionTree_clf = grid_search.best_estimator_
    
    # Si estrae solo il miglior risultato per questa tecnica
    best_params = grid_search.best_params_
    
    # Si definiscono i risultati per ogni tecnica, profondità e criterio
    result = {
        "techniques_list": technique,                                   # Si mantiene la lista di tecniche utilizzate per comodità
        "techniques": ', '.join(technique),                             # Si converte la lista di tecniche in stringa per migliore visualizzazione
        "max_depth": best_params["classifier__max_depth"],              # Si salva la profondità
        "criterion": best_params["classifier__criterion"],              # Si salva il criterio
        "mean_test_score": grid_search.best_score_,                     # Si salva il valore medio dell'accuratezza bilanciata ottenuto dalle 5 iterazioni della cross validation
        "std_test_score": grid_search.cv_results_["std_test_score"][    # Si salva il valore medio della deviazione standard (utile per verificare i risultati della cross validation)
            grid_search.cv_results_["mean_test_score"].argmax()
        ]               
    }
        
    # Si aggiunge il miglior risultato ai risultati finali
    results_decisionTrees.append(result)
    
    # Si libera la memoria
    del grid_search
        
# Si trasforma result in un pandas DataFrame
results_decisionTrees = pd.DataFrame(results_decisionTrees)

# Si restituisce il miglior risultato per ogni tecnica 
print("\nMigliori configurazioni per ogni tecnica:")
print(results_decisionTrees[["techniques", "max_depth", "criterion", "mean_test_score", "std_test_score"]].sort_values("mean_test_score", ascending=False))

```
Migliori configurazioni per ogni tecnica:
                                   techniques  max_depth criterion  mean_test_score  std_test_score
                  normalization, oversampling          8      gini         0.922296        0.006918
                standardization, oversampling          9      gini         0.920509        0.003477
                                 oversampling         10      gini         0.920508        0.005482
                                                      10      gini         0.917425        0.006273
                                normalization         10      gini         0.917425        0.006273
                              standardization         10      gini         0.917425        0.006273
 standardization, agglomeration, oversampling         10      gini         0.904470        0.006552
               standardization, agglomeration         10   entropy         0.900220        0.007012
               standardization, undersampling          7   entropy         0.898660        0.003765
                 normalization, undersampling          7   entropy         0.898262        0.005138
                 normalization, agglomeration          8      gini         0.889432        0.001961
   normalization, agglomeration, oversampling          9      gini         0.889157        0.002140
standardization, agglomeration, undersampling          9      gini         0.884840        0.008821
                                undersampling          6      gini         0.879965        0.015209
  normalization, agglomeration, undersampling          7      gini         0.876228        0.005636
```

In [ ]:
best_decisionTree_clf

Si verificano i risultati del codice precedente:
- Per ogni configurazione trovata, si applicano le medesime tecniche di preprocessing e si addestra un DecisionTree con gli ipeparametri ottimali.
- Si esegue una valutazione delle performance sul test set.
- Si visualizzano le matrici di confusione per ogni configurazione.
- Si salvano i risultati della migliore configurazione. 

In [ ]:
from sklearn.metrics import (multilabel_confusion_matrix, 
                             ConfusionMatrixDisplay, 
                             balanced_accuracy_score, 
                             confusion_matrix
                            )

# Si crea la directory per i decision trees se non esiste
dir = "5 - DECISION TREES"
path_dir = os.path.join(os.getcwd(), dir)

if not os.path.exists(path_dir):
    os.makedirs(path_dir)

# Si crea una lista per contenere i vari risultati
result_decisionTrees = []

# Si itera lungo le righe delle configurazioni migliori 
for row in results_decisionTrees.itertuples():
    # Si salva la lista contenente le tecniche di pre processing 
    technique = row.techniques_list
    
    # Si divide il dataset iniziale in train e test con stratificazione
    # random_state = 21 ci garantisce che per ogni iterazione otteniamo 
    # sempre le stesse separazioni
    X_train, X_test, y_train, y_test = combine_preprocessing(X, y, technique)
    
    # Si salva la profondità associata   
    max_depth = row.max_depth
    
    # Si salva il criterio associato
    criterion = row.criterion
    
    # Si definisce DecisionTreeClassifier con gli iperparametri definiti
    clf = DecisionTreeClassifier(max_depth=max_depth, criterion=criterion, random_state=21)
    
    # Si addestra il modello DecisionTreeClassifier sul train set
    clf.fit(X_train, y_train)
    
    # Si salvano le predizioni effettuate
    y_pred = clf.predict(X_test)
    
    # Si salva l'accuratezza bilanciata del modello
    balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
    
    # Si salvano i vari risultati d'interesse
    result = {
        "techniques_list": technique, 
        "technique": ', '.join(technique) if technique else 'no techniques',
        "accuracy": balanced_accuracy,
        "max_depth": max_depth,
        "criterion": criterion
    }
    
    # Si aggiungono ai risultati finali
    result_decisionTrees.append(result)
    
    # Si definiscono le matrici binarie (1 vs All)
    mcm = multilabel_confusion_matrix(y_test, y_pred)
    
    # Si salvano i nomi delle classi
    class_names = clf.classes_
    
    # Si crea una figura con 3 righe e 3 colonne di subplot
    fig, axes = plt.subplots(3, 3, figsize=(15, 15))
    
    # Si inserisce il titolo personalizzato
    fig.suptitle(f"Correlation Matrix Decision Trees\nTechniques: {', '.join(technique) if technique else 'no techniques'}\nMax_depth: {max_depth}\nCriterion: {criterion}\nAccuracy: {balanced_accuracy}")
    
    # Si trasforma la matrice 3x3 in un array 1D
    axes = axes.flatten()

    # Si itera per ogni coppia matrice - classe corrispondente
    for i, (matrix, class_name) in enumerate(zip(mcm, class_names)):
        
        # Si salvano i valori True Negative (tn) False Positive (fp)
        # False Negative (fn) e True Positive (tp)
        tn, fp = matrix[0]
        fn, tp = matrix[1]

        # Si calcola l'accuratezza per ogni classe
        accuracy = (tp + tn) / (tp + tn + fp + fn)
        
        # Si mostra il seguente messaggio nella matrice di correlazione
        display_labels = [f"Non-{class_name}", f"{class_name}"]

        # Si visualizza la matrice di ogni classe con ConfusionMatrixDisplay
        disp = ConfusionMatrixDisplay(confusion_matrix=matrix, display_labels=display_labels)
        disp.plot(ax=axes[i], cmap="Reds")
        axes[i].set_title(f"{class_name} vs Resto\nAccuracy: {accuracy}")

    # Si visualizza la matrice di confusione generale normalizzata 
    # si vedono le percentuali di corretta previsione e dove il modello fa confusione
    cm = confusion_matrix(y_test, y_pred, normalize="true")
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
    disp.plot(ax=axes[7], cmap="Reds", values_format=".2f")
    axes[7].set_title(f"Confusion Matrix Normalized")
    axes[7].set_xticklabels(class_names, rotation=90)
    
    # Non si visualizza l'ultimo subplot disponibile
    axes[8].set_visible(False)
    
    # Si salva l'immagine nell'apposita cartella
    plt.figure(fig.number)
    plt.tight_layout()
    plt.savefig(f"{dir}/{'_'.join(technique) if technique else 'no_techniques'}_confusion_matrix")
    plt.close(fig)
    
# Si converte results in un pandas DataFrame e si ordinano i risultati
result_decisionTrees = pd.DataFrame(result_decisionTrees)
print(result_decisionTrees.sort_values("accuracy", ascending=False))

# Si ricava la migliore accuracy e si stamapa a video
print("\nMiglior configurazione:")
best_idx = result_decisionTrees['accuracy'].idxmax()
best_config_decisionTree = result_decisionTrees.iloc[[best_idx]]
print(best_config_decisionTree[["technique", "accuracy", "max_depth", "criterion"]])

```
Miglior configurazione:
                    technique  accuracy  max_depth criterion
standardization, oversampling  0.926019          9      gini
```

In [ ]:
# Si salva la miglior configurazione trovata
best_configs_list.append(("DecisionTree", best_config_decisionTree))

### K-Nearest Neighbors

L'algoritmo **k-NN** è un algoritmo di machine learning che definisce la classe di un punto sulla base delle classi delle **k osservazioni nelle vicinanze**. Viene definito come un algoritmo di *apprendimeto lazy* in quanto non costruisce un modello "esplicito" durante la fase di addestramento. Inoltre presenta una serie di metriche di distanza che si occupano di determinare come viene calcolata la similarità tra occorrenze.

I punti di forza del k-NN sono la semplicità concettuale e implementativa, ma può diventare computazionalmente costoso con dataset di grandi dimensioni, inoltre ha la necessità che i dati appartengano alla stessa scala.

La libreria *scikit-learn* mette a disposizione la classe [*KNeighborsClassifier*](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html) per implementare l'algoritmo k-Nearest Neighbour. Quest'ultimo utilizza la distanza euclidea by default (come caso speciale della distanza minkowski con p = 2) e k = 5.

Si effettua un tuning degli iperparametri tramite *GridSearchCV* per definire la miglior combinazione tra numero di k vicini da considerare, pesi per la distanza e distanza da utilizzare.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# skf = StratifiedKFold è già stato definito precedentemente

# Si definisce una lista per contenere i risultati
results_knn = []

# Si definisce il classificatore KNeighborsClassifier su cui effettuare
# il tuning degli iperparametri. Essendo deterministico non presenta random state
clf = KNeighborsClassifier(n_jobs=-1)

# Si definisce una lista dei vicini da testare durante la cross validation
k_neighbors = list(range(1, 21)) # 1 - 20

# Si definisce una lista dei parametri per la ponderazione da apllicare
# ai vicini
weights = ['uniform', 'distance']

# Si definisce una lisa di metriche di distanze da utilizzare 
metric = ['euclidean', 'manhattan', 'chebyshev']

# Si divide il dataset iniziale in train e test con stratificazione 
# ovvero le occorrenze delle classi sono automaticamente bilanciate
# tra train e test (random state garantisce la riproducibilità)
X_train, _, y_train, _ = train_test_split(X, y, test_size=0.2, random_state=21, stratify=y)

# Si inizializzano le due variabili utilizzate successivamente per salvare il miglior
# classificatore ttrovato tramite la grid search e cross validation
best_knn_score = -1
best_knn_clf = None

# Si itera per tutte le combinazioni di tecniche di pre processing possibili 
# e precedentemente definite
for technique in techniques:
    
    # Si definisce una lista per contenere i vari step per la cross validation
    steps = []
    
    # Se la tecnica specificata tra virgolette è presente nella lista
    # technique si aggiunge tra gli step della cross validation
    if "standardization" in technique:
        steps.append(("scaler", scaler))
        
        # Si salva il numero corretto di cluster per la standardizzazione 
        n_clusters = n_clusters_std
        
    elif "normalization" in technique:
        steps.append(("normalizer", normalizer))
        
        # Si salva il numero corretto di cluster per la normalizzazione
        n_clusters = n_clusters_std
        
    if "agglomeration" in technique:
        
        # Si definisce lo step dell'agglomerazione di feature con gli stessi 
        # parametri usati precedentemente 
        steps.append(("agglomeration", FeatureAgglomeration(
            n_clusters=n_clusters,
            metric="euclidean",
            linkage="ward"
        )))
        
    if "undersampling" in technique:
        steps.append(("undersampler", cc))
        
    elif "oversampling" in technique:
        steps.append(("oversampler", smote))
    
    # Si aggiunge il classificatore KNeighborsClassifier come ultimo step
    # della cross validation
    steps.append(("classifier", clf))
    
    # Si crea la pipeline di step 
    # si utilizza ImbPipeline invece di Pipeline in quanto accetta 
    # le tecniche di sampling (oversampling/undersampling)
    pipeline = ImbPipeline(steps)
    
    # Si definisce il parametro di GridSearchCV per la ricerca degli
    # iperparametri migliori
    param_grid = {
        "classifier__n_neighbors": k_neighbors,
        "classifier__weights": weights,
        "classifier__metric": metric
    }
    
    # Si definisce GridSearchCV, la pipeline da seguire, i parametri,
    # la tecnica per la cross validation (StratifiedKFold), lo scoring da 
    # utilizzare (balanced_accuracy in quanto il dataset è sbilanciato) e il
    # numero di core da utilizzare contemporaneamente
    grid_search = GridSearchCV(
        estimator=pipeline,
        param_grid=param_grid,
        cv=skf,
        scoring="balanced_accuracy",
        n_jobs=-1,
        verbose=1
    )
    
    # Si esegue la cross validation sul train set, per ogni tecnica si 
    # verifica quale combinazione di iperparametri ottiene l'accuratezza maggiore
    grid_search.fit(X_train, y_train)
    
    # Si verifica il risultato del modello, se migliore del precedente si salva il corrente
    if grid_search.best_score_ > best_knn_score:
        best_knn_score = grid_search.best_score_
        best_knn_clf = grid_search.best_estimator_
    
    # Si estrae solo il miglior risultato per questa tecnica
    best_params = grid_search.best_params_
    
    # Si definiscono i risultati per ogni tecnica, profondità e criterio
    result = {
        "techniques_list": technique,                                   # Si mantiene la lista di tecniche utilizzate per comodità
        "techniques": ', '.join(technique),                             # Si converte la lista di tecniche in stringa per migliore visualizzazione
        "n_neighbors": best_params["classifier__n_neighbors"],          # Si salva il valore di k
        "weights": best_params["classifier__weights"],                  # Si salva il parametro peso
        "metric": best_params["classifier__metric"],                    # Si salva la metrica
        "mean_test_score": grid_search.best_score_,                     # Si salva il valore medio dell'accuratezza bilanciata ottenuto dalle 5 iterazioni della cross validation
        "std_test_score": grid_search.cv_results_["std_test_score"][    # Si salva la deviazione standard del miglior punteggio
            grid_search.cv_results_["mean_test_score"].argmax()
        ]
    }
        
    # Si aggiungono ai risultati finali
    results_knn.append(result)
    
    # Si libera la memoria
    del grid_search
        
# Si trasforma result in un pandas DataFrame
results_knn = pd.DataFrame(results_knn)

# Si restituisce la combinazione dei risultati migliori per ogni tecnica
print("\nMigliori configurazioni per ogni tecnica:")
print(results_knn[["techniques", "n_neighbors", "weights", "metric", "mean_test_score", "std_test_score"]].sort_values("mean_test_score", ascending=False)) 

```
Migliori configurazioni per ogni tecnica:
                                   techniques  n_neighbors   weights     metric  mean_test_score  std_test_score
                standardization, oversampling           14  distance  euclidean         0.935410        0.005006
               standardization, undersampling            9  distance  manhattan         0.934222        0.002982
                              standardization            6  distance  euclidean         0.933960        0.003315
                 normalization, undersampling           12  distance  euclidean         0.931810        0.002194
                                normalization            8  distance  euclidean         0.931211        0.002908
                  normalization, oversampling           12  distance  chebyshev         0.931021        0.004047
standardization, agglomeration, undersampling           14  distance  manhattan         0.905852        0.007083
 standardization, agglomeration, oversampling           18  distance  euclidean         0.905194        0.002247
               standardization, agglomeration            8  distance  euclidean         0.902804        0.001674
                 normalization, agglomeration           17   uniform  euclidean         0.897193        0.004668
  normalization, agglomeration, undersampling           19  distance  manhattan         0.897127        0.007467
   normalization, agglomeration, oversampling           19  distance  manhattan         0.895566        0.004555
                                 oversampling            6  distance  manhattan         0.786889        0.004193
                                                         7  distance  manhattan         0.786682        0.005773
                                undersampling            4  distance  manhattan         0.784426        0.012981
```

In [ ]:
best_knn_clf

Si verificano i risultati del codice precedente:
- Per ogni configurazione trovata, si applicano le medesime tecniche di preprocessing e si addestra un algoritmo KNN con gli ipeparametri ottimali.
- Si esegue una valutazione delle performance sul test set.
- Si visualizzano le matrici di confusione per ogni configurazione.
- Si salvano i risultati della migliore configurazione. 

In [ ]:
# Si crea la directory per KNN se non esiste
dir = "6 - KNN"
path_dir = os.path.join(os.getcwd(), dir)

if not os.path.exists(path_dir):
    os.makedirs(path_dir)

# Si crea una lista per contenere i vari risultati
result_knn = []

# Si itera lungo le righe delle configurazioni migliori 
for row in results_knn.itertuples():
    
    # Si salva la lista contenente le tecniche di pre processing 
    technique = row.techniques_list
    
    # Si divide il dataset iniziale in train e test con stratificazione
    # random_state = 21 ci garantisce che per ogni iterazione otteniamo 
    # sempre le stesse separazioni e si combinano le varie tecniche di preprocessing
    X_train, X_test, y_train, y_test = combine_preprocessing(X, y, technique)
    
    # Si salva il valore di n (vicini) 
    k_neighbors = row.n_neighbors
    
    # Si salva il parametro peso 
    weights = row.weights
    
    # Si salva la metrica considerata migliore
    metric = row.metric
    
    # Si definisce KNeighborsClassifier con gli iperparametri definiti
    clf = KNeighborsClassifier(n_neighbors=k_neighbors, weights=weights, metric=metric, n_jobs=-1)
    
    # Si addestra il modello KNeighborsClassifier sul train set
    clf.fit(X_train, y_train)
    
    # Si salvano le predizioni effettuate
    y_pred = clf.predict(X_test)
    
    # Si salva l'accuratezza bilanciata del modello
    balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
    
    # Si salvano i vari risultati d'interesse
    result = {
        "techniques_list": technique, 
        "technique": ', '.join(technique) if technique else 'no techniques',
        "accuracy": balanced_accuracy,
        "k_neighbors": k_neighbors,
        "weights": weights,
        "metric": metric
    }
    
    # Si aggiungono ai risultati finali
    result_knn.append(result)
    
    # Si definiscono le matrici binarie (1 vs All)
    mcm = multilabel_confusion_matrix(y_test, y_pred)
    
    # Si salvano i nomi delle classi
    class_names = clf.classes_
    
    # Si crea una figura con 3 righe e 3 colonne di subplot
    fig, axes = plt.subplots(3, 3, figsize=(15, 15))
    
    # Si inserisce il titolo personalizzato
    fig.suptitle(f"Correlation Matrix KNN\nTechniques: {', '.join(technique) if technique else 'no techniques'}\nK Neighbors: {k_neighbors}\nMetric: {metric} with {weights} weights\nAccuracy: {balanced_accuracy}")
    
    # Si trasforma la matrice 3x3 in un array 1D
    axes = axes.flatten()

    # Si itera per ogni coppia matrice - classe corrispondente
    for i, (matrix, class_name) in enumerate(zip(mcm, class_names)):
        
        # Si salvano i valori True Negative (tn) False Positive (fp)
        # False Negative (fn) e True Positive (tp)
        tn, fp = matrix[0]
        fn, tp = matrix[1]

        # Si calcola l'accuratezza per ogni classe
        accuracy = (tp + tn) / (tp + tn + fp + fn)
        
        # Si mostra il seguente messaggio nella matrice di correlazione
        display_labels = [f"Non-{class_name}", f"{class_name}"]

        # Si visualizza la matrice di ogni classe con ConfusionMatrixDisplay
        disp = ConfusionMatrixDisplay(confusion_matrix=matrix, display_labels=display_labels)
        disp.plot(ax=axes[i], cmap="Reds")
        axes[i].set_title(f"{class_name} vs Resto\nAccuracy: {accuracy}")

    # Si visualizza la matrice di confusione generale normalizzata 
    # si vedono le percentuali di corretta previsione e dove il modello fa confusione
    cm = confusion_matrix(y_test, y_pred, normalize="true")
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
    disp.plot(ax=axes[7], cmap="Reds", values_format=".2f")
    axes[7].set_title(f"Confusion Matrix Normalized")
    axes[7].set_xticklabels(class_names, rotation=90)
    
    # Non si visualizza l'ultimo subplot disponibile
    axes[8].set_visible(False)
    
    # Si salva l'immagine nell'apposita cartella
    plt.figure(fig.number)
    plt.tight_layout()
    plt.savefig(f"{dir}/{'_'.join(technique) if technique else 'no_techniques'}_confusion_matrix")
    plt.close(fig)
    
# Si converte results in un pandas DataFrame e si ordinano i risultati
result_knn = pd.DataFrame(result_knn)
print(result_knn.sort_values("accuracy", ascending=False))

# Si ricava la migliore accuracy e si stamapa a video
print("\nMiglior configurazione:")
best_idx = result_knn['accuracy'].idxmax()
best_config_knn = result_knn.iloc[[best_idx]]
print(best_config_knn[["technique", "accuracy", "k_neighbors", "weights", "metric"]])

```
Miglior configurazione:
                    technique  accuracy  k_neighbors   weights     metric
standardization, oversampling  0.936496           14  distance  euclidean
```

In [ ]:
# Si salva la miglior configurazione trovata
best_configs_list.append(("KNN", best_config_knn))

### Naive Bayes

I Naive Bayes sono una famiglia di algoritmi di classificazione probabilistici basati sul **teorema di Bayes** con assunzione di indipendenza tra le feature. Questi algoritmi calcolano la *probabilità di appartenenza a ciascuna classe* e assegnano l'osservazione alla classe con la *probabilità più elevata*.

L'assunzione di indipendenza condizionale tra le feature (anche se spesso è una semplificazione della realtà) rende questi algoritmi computazionalmente efficienti.

I punti di frza dei classificatori Naive Bayes sono la semplicità concettuale, la velocità di addestramento e predizione, la robustezza al rumore e la capacità di gestire dati ad alta dimensionalità. 

La libreria *scikit-learn* implementa diverse varianti di Naive Bayes, ciascuna adatta a diverse distribuzioni di dati. Tra queste, **GaussianNB** è specifica per feature continue che si assume seguano una distribuzione normale all'interno di ciascuna classe. [*GaussianNB*](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html) stima media e varianza per ogni feature condizionata alla classe, e utilizza la funzione di densità di probabilità gaussiana per calcolare le probabilità durante la fase predittiva.

Si effettua un tuning degli iperparametri tramite *GridSearchCV* per definire la miglior *var_smoothing* (per stabilizzare il calcolo). Inoltre si verifica se i risultati migliorano impostando *priors* (ovvero le probabilità a priori delle classi) con valori inversi alla frequenza delle classi.

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.utils.class_weight import compute_class_weight

# skf = StratifiedKFold è già stato definito precedentemente

# Si definisce una lista per contenere i risultati
results_naiveBayes = []

# Si definisce il classificatore GaussianNB su cui effettuare
# il tuning degli iperparametri. Essendo deterministico non presenta random state
clf = GaussianNB()

# Si definisce una lista delle var_smoothing da testare (scala logaritmica, usiamo logspace)
var_smoothing = np.logspace(-17, -9, 9) # [1.e-17, 1.e-16, 1.e-15, 1.e-14, 1.e-13, 1.e-12, 1.e-11, 1.e-10, 1.e-09]7]

# Si divide il dataset iniziale in train e test con stratificazione 
# ovvero le occorrenze delle classi sono automaticamente bilanciate
# tra train e test (random state garantisce la riproducibilità)
X_train, _, y_train, _ = train_test_split(X, y, test_size=0.2, random_state=21, stratify=y)

# Si calcolano i pesi inversamente proporzionali
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)

# Si normalizzano per ottenere probabilità che sommano a 1
weights = class_weights / class_weights.sum()

# Si definisce una lista delle probabilità a priori da testare nel modello
priors = [None, weights]

# Si inizializzano le due variabili utilizzate successivamente per salvare il miglior
# classificatore ttrovato tramite la grid search e cross validation
best_naiveBayes_score = -1
best_naiveBayes_clf = None

# Si itera per tutte le combinazioni di tecniche di pre processing possibili 
# e precedentemente definite
for technique in techniques:
    
    # Si definisce una lista per contenere i vari step per la cross validation
    steps = []
    
    # Se la tecnica specificata tra virgolette è presente nella lista
    # technique si aggiunge tra gli step della cross validation
    if "standardization" in technique:
        steps.append(("scaler", scaler))
        
        # Si salva il numero corretto di cluster per la standardizzazione 
        n_clusters = n_clusters_std
        
    elif "normalization" in technique:
        steps.append(("normalizer", normalizer))
        
        # Si salva il numero corretto di cluster per la normalizzazione
        n_clusters = n_clusters_std
        
    if "agglomeration" in technique:
        
        # Si definisce lo step dell'agglomerazione di feature con gli stessi 
        # parametri usati precedentemente 
        steps.append(("agglomeration", FeatureAgglomeration(
            n_clusters=n_clusters,
            metric="euclidean",
            linkage="ward"
        )))
        
    if "undersampling" in technique:
        steps.append(("undersampler", cc))
        
    elif "oversampling" in technique:
        steps.append(("oversampler", smote))
    
    # Si aggiunge il classificatore KNeighborsClassifier come ultimo step
    # della cross validation
    steps.append(("classifier", clf))
    
    # Si crea la pipeline di step 
    # si utilizza ImbPipeline invece di Pipeline in quanto accetta 
    # le tecniche di sampling (oversampling/undersampling)
    pipeline = ImbPipeline(steps)
    
    # Si definisce il parametro di GridSearchCV per la ricerca degli
    # iperparametri migliori
    param_grid = {
        "classifier__var_smoothing": var_smoothing,
        "classifier__priors": priors
    }
    
    # Si definisce GridSearchCV, la pipeline da seguire, i parametri,
    # la tecnica per la cross validation (StratifiedKFold), lo scoring da 
    # utilizzare (balanced_accuracy in quanto il dataset è sbilanciato) e il
    # numero di core da utilizzare contemporaneamente
    grid_search = GridSearchCV(
        estimator=pipeline,
        param_grid=param_grid,
        cv=skf,
        scoring="balanced_accuracy",
        n_jobs=-1,
        verbose=1
    )
    
    # Si esegue la cross validation sul train set, per ogni tecnica si 
    # verifica quale combinazione di iperparametri ottiene l'accuratezza maggiore
    grid_search.fit(X_train, y_train)
    
    # Si verifica il risultato del modello, se migliore del precedente si salva il corrente
    if grid_search.best_score_ > best_naiveBayes_score:
        best_naiveBayes_score = grid_search.best_score_
        best_naiveBayes_clf = grid_search.best_estimator_
    
    # Si estrae solo il miglior risultato per questa tecnica
    best_params = grid_search.best_params_
    
    # Si definiscono i risultati per ogni tecnica, profondità e criterio
    result = {
        "techniques_list": technique,                                   # Si mantiene la lista di tecniche utilizzate per comodità
        "techniques": ', '.join(technique),                             # Si converte la lista di tecniche in stringa per migliore visualizzazione
        "var_smoothing": best_params["classifier__var_smoothing"],      # Si salva il valore di var_smoothing
        "priors": best_params["classifier__priors"],                    # Si salva il parametro probabilità a priori
        "mean_test_score": grid_search.best_score_,                     # Si salva il valore medio dell'accuratezza bilanciata ottenuto dalle 5 iterazioni della cross validation
        "std_test_score": grid_search.cv_results_["std_test_score"][    # Si salva la deviazione standard del miglior punteggio
            grid_search.cv_results_["mean_test_score"].argmax()
        ]
    }
        
    # Si aggiungono ai risultati finali
    results_naiveBayes.append(result)

    # Si libera la memoria
    del grid_search
    
# Si trasforma result in un pandas DataFrame
results_naiveBayes = pd.DataFrame(results_naiveBayes)

# Si restituisce la combinazione dei risultati migliori per ogni tecnica
print("\nMigliori configurazioni per ogni tecnica:")
print(results_naiveBayes[["techniques", "var_smoothing", "priors", "mean_test_score", "std_test_score"]].sort_values("mean_test_score", ascending=False)) 

```
Migliori configurazioni per ogni tecnica:
                                   techniques  var_smoothing    priors  mean_test_score  std_test_score
                                 oversampling   1.000000e-16   Weights         0.911079        0.004048
                                                1.000000e-15      None         0.909188        0.003429
                  normalization, oversampling   1.000000e-17   Weights         0.908160        0.002980
                standardization, oversampling   1.000000e-17   Weights         0.907771        0.002996
                              standardization   1.000000e-17      None         0.907439        0.003649
                                normalization   1.000000e-17      None         0.907439        0.003649
                                undersampling   1.000000e-17      None         0.904302        0.002789
                 normalization, undersampling   1.000000e-17   Weights         0.902195        0.004325
               standardization, undersampling   1.000000e-17   Weights         0.902069        0.004143
 standardization, agglomeration, oversampling   1.000000e-17   Weights         0.882559        0.005692
               standardization, agglomeration   1.000000e-17   Weights         0.881982        0.005774
                 normalization, agglomeration   1.000000e-17      None         0.881065        0.004722
   normalization, agglomeration, oversampling   1.000000e-17   Weights         0.880719        0.005252
standardization, agglomeration, undersampling   1.000000e-17      None         0.878343        0.005376
  normalization, agglomeration, undersampling   1.000000e-17   Weights         0.878152        0.006201
```

In [ ]:
best_naiveBayes_clf

Si verificano i risultati del codice precedente:
- Per ogni configurazione trovata, si applicano le medesime tecniche di preprocessing e si addestra un algoritmo NaiveBayes (GaussianNB) con gli ipeparametri ottimali.
- Si esegue una valutazione delle performance sul test set.
- Si visualizzano le matrici di confusione per ogni configurazione.
- Si salvano i risultati della migliore configurazione. 

In [ ]:
# Si crea la directory per la Naive Bayes Gaussiana se non esiste
dir = "7 - NAIVE BAYES"
path_dir = os.path.join(os.getcwd(), dir)

if not os.path.exists(path_dir):
    os.makedirs(path_dir)

# Si crea una lista per contenere i vari risultati
result_naiveBayes = []

# Si itera lungo le righe delle configurazioni migliori 
for row in results_naiveBayes.itertuples():
    
    # Si salva la lista contenente le tecniche di pre processing 
    technique = row.techniques_list
    
    # Si divide il dataset iniziale in train e test con stratificazione
    # random_state = 21 ci garantisce che per ogni iterazione otteniamo 
    # sempre le stesse separazioni e si combinano le varie tecniche di preprocessing
    X_train, X_test, y_train, y_test = combine_preprocessing(X, y, technique)
    
    # Si salva il valore di var_smoothing
    var_smoothing = row.var_smoothing
    
    # Si salva il parametro priors
    priors = row.priors
    
    # Si definisce GaussianNB con gli iperparametri definiti
    clf = GaussianNB(var_smoothing=var_smoothing, priors=priors)
    
    # Si addestra il modello GaussianNB sul train set
    clf.fit(X_train, y_train)
    
    # Si salvano le predizioni effettuate
    y_pred = clf.predict(X_test)
    
    # Si salva l'accuratezza bilanciata del modello
    balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
    
    # Si salvano i vari risultati d'interesse
    result = {
        "techniques_list": technique, 
        "technique": ', '.join(technique) if technique else 'no techniques',
        "accuracy": balanced_accuracy,
        "var_smoothing": var_smoothing,
        "priors": priors
    }
    
    # Si aggiungono ai risultati finali
    result_naiveBayes.append(result)
    
    # Si definiscono le matrici binarie (1 vs All)
    mcm = multilabel_confusion_matrix(y_test, y_pred)
    
    # Si salvano i nomi delle classi
    class_names = clf.classes_
    
    # Si crea una figura con 3 righe e 3 colonne di subplot
    fig, axes = plt.subplots(3, 3, figsize=(15, 15))
    
    # Si inserisce il titolo personalizzato
    fig.suptitle(f"Correlation Matrix NaiveBayes\nTechniques: {', '.join(technique) if technique else 'no techniques'}\nVar Smoothing: {var_smoothing}\nPriors: {priors}\nAccuracy: {balanced_accuracy}")
    
    # Si trasforma la matrice 3x3 in un array 1D
    axes = axes.flatten()

    # Si itera per ogni coppia matrice - classe corrispondente
    for i, (matrix, class_name) in enumerate(zip(mcm, class_names)):
        
        # Si salvano i valori True Negative (tn) False Positive (fp)
        # False Negative (fn) e True Positive (tp)
        tn, fp = matrix[0]
        fn, tp = matrix[1]

        # Si calcola l'accuratezza per ogni classe
        accuracy = (tp + tn) / (tp + tn + fp + fn)
        
        # Si mostra il seguente messaggio nella matrice di correlazione
        display_labels = [f"Non-{class_name}", f"{class_name}"]

        # Si visualizza la matrice di ogni classe con ConfusionMatrixDisplay
        disp = ConfusionMatrixDisplay(confusion_matrix=matrix, display_labels=display_labels)
        disp.plot(ax=axes[i], cmap="Reds")
        axes[i].set_title(f"{class_name} vs Resto\nAccuracy: {accuracy}")

    # Si visualizza la matrice di confusione generale normalizzata 
    # si vedono le percentuali di corretta previsione e dove il modello fa confusione
    cm = confusion_matrix(y_test, y_pred, normalize="true")
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
    disp.plot(ax=axes[7], cmap="Reds", values_format=".2f")
    axes[7].set_title(f"Confusion Matrix Normalized")
    axes[7].set_xticklabels(class_names, rotation=90)
    
    # Non si visualizza l'ultimo subplot disponibile
    axes[8].set_visible(False)
    
    # Si salva l'immagine nell'apposita cartella
    plt.figure(fig.number)
    plt.tight_layout()
    plt.savefig(f"{dir}/{'_'.join(technique) if technique else 'no_techniques'}_confusion_matrix")
    plt.close(fig)
    
# Si converte results in un pandas DataFrame e si ordinano i risultati
result_naiveBayes = pd.DataFrame(result_naiveBayes)
print(result_naiveBayes.sort_values("accuracy", ascending=False))

# Si ricava la migliore accuracy e si stamapa a video
print("\nMiglior configurazione:")
best_idx = result_naiveBayes['accuracy'].idxmax()
best_config_naiveBayes = result_naiveBayes.iloc[[best_idx]]
print(best_config_naiveBayes[["technique", "accuracy", "var_smoothing", "priors"]])

```
Miglior configurazione:
   technique  accuracy  var_smoothing  priors
oversampling   0.90995   1.000000e-16 Weights
```

In [ ]:
# Si salva la miglior configurazione trovata
best_configs_list.append(("NaiveBayes", best_config_naiveBayes))

### Support Vector Classifier (SVC)

SVC è un algoritmo di machine learning che cerca di trovare un *iperpiano ottimale per separare dati appartenenti a classi diverse*. Opera massimizzando il margine tra le classi, ovvero la distanza tra l'iperpiano di separazione e i punti più vicini di ciascuna classe (chiamati vettori di supporto).

L'algoritmo utilizza il "kernel trick" che permette di trasformare implicitamente i dati in uno spazio dimensionale superiore, dove potrebbero diventare linearmente separabili. SVC supporta diversi tipi di kernel.

Tra i punti di forza di SVC c'è l'efficacia in spazi ad alta dimensionalità, tuttavia, non scala bene con dataset molto grandi e richiede un'attenta ottimizzazione degli iperparametri.

La libreria *scikit-learn* implementa la classe [*SVC*](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html). By default utilizza kernel RBF con gamma='scale' e C=1.0, rendendo il modello abbastanza flessibile ma suscettibile a overfitting in alcuni casi.

Si effettua un tuning degli iperparametri tramite *GridSearchCV* per trovare la migliore combinazione di:
- **Kernel** (*linear* o *rbf*): funzione che trasforma i dati in uno spazio dove diventano più facilmente separabili.
- **C**: definisce quanto il modello deve evitare errori di classificazione. 
- **Gamma**: determina quanto lontano arriva l'influenza di ogni punto.
- **Class weight** (*balaced*): assegna l'importanza agli errori di ciascuna classe (utile in caso di dataset sbilanciato).

Si noti che, essendo *SVC* una tecnica molto costosa computazionalmente e dal punto di vista della gestione della memoria, si effettua un tuning degli iperparametri tramite *GridSearchCV* su un subset del dataset (10% del dataset originale con startificazione).

In [ ]:
from sklearn.svm import SVC

# Si definisce una lista per contenere i risultati
results_svc = []

# Si definisce il classificatore SVC su cui effettuare il tuning degli iperparametri.
# Si imposta random state per la riproducibilità
clf = SVC(random_state=21)

# Si definisce una lista dei parametri di C da testare 
# tipicamente su scale logaritmiche, si usa logspace di numpy
c = np.logspace(-1, 1, 3) # [ 0.1, 1, 10]

# Si definisce una lista dei parametri di gamma da testare
# tipicamente su scale logaritmiche, si usa logspace di numpy
gamma = ['scale', 'auto'] + list(np.logspace(-1, 0, 2)) # ['scale', "auto", 0.1 , 1]

# Si definisce la metrica per la valutazione dei pesi associati alle classi
class_weight = [None, 'balanced']

# Si definiscono grid diversi per kernel diversi (in quanto solo rbf considera gamma come parametro)
linear_params = {
    "classifier__kernel": ['linear'], 
    "classifier__C": c, 
    "classifier__class_weight": class_weight
}
rbf_params = {
    "classifier__kernel": ['rbf'], 
    "classifier__C": c, 
    "classifier__gamma": gamma,
    "classifier__class_weight": class_weight
    }

# Si definisce un subset del dataset originale
X_subset, _, y_subset, _ = train_test_split(X, y, train_size=0.1, random_state=21, stratify=y)

# Si divide il subset del dataset originale in train e test con stratificazione 
# ovvero le occorrenze delle classi sono automaticamente bilanciate
# tra train e test (random state garantisce la riproducibilità)
X_train, _, y_train, _ = train_test_split(X_subset, y_subset, test_size=0.2, random_state=21, stratify=y_subset)

# Si inizializzano le due variabili utilizzate successivamente per salvare il miglior
# classificatore ttrovato tramite la grid search e cross validation
best_svc_score = -1
best_svc_clf = None

# Si itera per tutte le combinazioni di tecniche di pre processing possibili 
# e precedentemente definite
for technique in techniques:
    
    # Si definisce una lista per contenere i vari step per la cross validation
    steps = []
    
    # Se la tecnica specificata tra virgolette è presente nella lista
    # technique si aggiunge tra gli step della cross validation
    if "standardization" in technique:
        steps.append(("scaler", scaler))
        
        # Si salva il numero corretto di cluster per la standardizzazione 
        n_clusters = n_clusters_std
        
    elif "normalization" in technique:
        steps.append(("normalizer", normalizer))
        
        # Si salva il numero corretto di cluster per la normalizzazione
        n_clusters = n_clusters_std
        
    if "agglomeration" in technique:
        
        # Si definisce lo step dell'agglomerazione di feature con gli stessi 
        # parametri usati precedentemente 
        steps.append(("agglomeration", FeatureAgglomeration(
            n_clusters=n_clusters,
            metric="euclidean",
            linkage="ward"
        )))
        
    if "undersampling" in technique:
        steps.append(("undersampler", cc))
        
    elif "oversampling" in technique:
        steps.append(("oversampler", smote))
    
    # Si aggiunge il classificatore SVC come ultimo step della cross validation
    steps.append(("classifier", clf))
    
    # Si crea la pipeline di step 
    # si utilizza ImbPipeline invece di Pipeline in quanto accetta 
    # le tecniche di sampling (oversampling/undersampling)
    pipeline = ImbPipeline(steps)

    # Si definisce il parametro param_grid come combinazione degli iperparametri da valutare
    param_grid = [linear_params, rbf_params]
    
    # Si definisce GridSearchCV, la pipeline da seguire, i parametri,
    # la tecnica per la cross validation (StratifiedKFold), lo scoring da 
    # utilizzare (balanced_accuracy in quanto il dataset è sbilanciato) e il
    # numero di core da utilizzare contemporaneamente
    grid_search = GridSearchCV(
        estimator=pipeline,
        param_grid=param_grid,
        cv=skf,
        scoring="balanced_accuracy",
        n_jobs=-1,
        verbose=1
    )

    # Si esegue la cross validation sul train set, per ogni tecnica si 
    # verifica quale combinazione di iperparametri ottiene l'accuratezza maggiore
    grid_search.fit(X_train, y_train)
    
    # Si verifica il risultato del modello, se migliore del precedente si salva il corrente
    if grid_search.best_score_ > best_svc_score:
        best_svc_score = grid_search.best_score_
        best_svc_clf = grid_search.best_estimator_
        
    # Si estrae solo il miglior risultato per questa tecnica
    best_params = grid_search.best_params_
    
    # Si crea il dizionario con i risultati migliori per questa tecnica
    result = {
        "techniques_list": technique,                                        # Si mantiene la lista di tecniche utilizzate per comodità
        "techniques": ', '.join(technique),                                  # SI converte in stringa la lista delle tecniche utilizzate
        "kernel": best_params["classifier__kernel"],                         # Si salva il Kernel migliore
        "C": best_params["classifier__C"],                                   # Si salva il parametro C migliore
        "gamma": best_params.get("classifier__gamma", "None"),               # Si salva il parametro Gamma migliore (se presente)
        "class_weight": best_params["classifier__class_weight"],             # Si salva il Class weight migliore
        "mean_test_score": grid_search.best_score_,                          # Si salva lo score
        "std_test_score": grid_search.cv_results_["std_test_score"][         # Si salva la deviazione standard del miglior punteggio
            grid_search.cv_results_["mean_test_score"].argmax()
        ]
    }
    
    # Si aggiunge solo il miglior risultato ai risultati finali
    results_svc.append(result)
    
    # Si libera la memoria
    del grid_search
    
# Si trasforma results in un pandas DataFrame
results_svc = pd.DataFrame(results_svc)

# Si restituiscono i risultati ordinati per performance
print("\nMigliori configurazioni per ogni tecnica:")
print(results_svc[["techniques", "kernel", "C", "gamma", "class_weight", "mean_test_score", "std_test_score"]].sort_values("mean_test_score", ascending=False))

```
Migliori configurazioni per ogni tecnica:
                                   techniques  kernel     C  gamma  class_weight  mean_test_score  std_test_score
                 normalization, undersampling     rbf  10.0  scale          None         0.949491        0.010041
                              standardization     rbf  10.0   0.01          None         0.948215        0.008798
                                normalization     rbf  10.0  scale          None         0.947795        0.010418
                  normalization, oversampling     rbf   1.0  scale          None         0.946671        0.005631
                standardization, oversampling  linear   0.1   None          None         0.946648        0.005566
               standardization, agglomeration  linear  10.0   None      balanced         0.945868        0.004841
               standardization, undersampling     rbf  10.0   0.01          None         0.945500        0.008319
 standardization, agglomeration, oversampling  linear  10.0   None          None         0.943577        0.003275
                                               linear   1.0   None          None         0.937080        0.006882
                                 oversampling  linear   1.0   None          None         0.935499        0.011211
standardization, agglomeration, undersampling     rbf  10.0  scale          None         0.930183        0.005209
   normalization, agglomeration, oversampling     rbf  10.0    1.0          None         0.919025        0.009025
                 normalization, agglomeration     rbf  10.0  scale      balanced         0.916079        0.008972
  normalization, agglomeration, undersampling     rbf  10.0  scale          None         0.913489        0.012633
                                undersampling  linear  10.0   None          None         0.910412        0.013463
```

In [ ]:
best_svc_clf

Si verificano i risultati del codice precedente:
- Per ogni configurazione trovata, si applicano le medesime tecniche di preprocessing e si addestra un algoritmo SVC con gli ipeparametri ottimali.
- Si esegue una valutazione delle performance sul test set.
- Si visualizzano le matrici di confusione per ogni configurazione.
- Si salvano i risultati della migliore configurazione. 

In [ ]:
# Si crea la directory per SVC se non esiste
dir = "8 - SVC"
path_dir = os.path.join(os.getcwd(), dir)

if not os.path.exists(path_dir):
    os.makedirs(path_dir)

# Si crea una lista per contenere i vari risultati
result_svc = []

# Si ridefinisce il medesimo subset per lavorare con il test set corretto
X_subset, _, y_subset, _ = train_test_split(X, y, train_size=0.1, random_state=21, stratify=y)

# Si itera lungo le righe delle configurazioni migliori 
for row in results_svc.itertuples():
    
    # Si salva la lista contenente le tecniche di pre processing 
    technique = row.techniques_list
    
    # Si divide il subset del dataset in train e test con stratificazione
    # random_state = 21 ci garantisce che per ogni iterazione otteniamo 
    # sempre le stesse separazioni e si combinano le varie tecniche di preprocessing
    X_train, X_test, y_train, y_test = combine_preprocessing(X_subset, y_subset, technique)
    
    # Si salva il valore d C
    c = row.C
    
    # Si salva il parametro gamma 
    gamma = row.gamma
    
    # Si salva il parametro peso
    class_weight = row.class_weight
    
    # Si salva il kernel utilizzato
    kernel = row.kernel
    
    # Si definisce SVC con gli iperparametri definiti
    if gamma == "None":
        clf = SVC(C=c, kernel=kernel, class_weight=class_weight, random_state=21)
    else:
        clf = SVC(C=c,kernel=kernel, gamma=gamma, class_weight=class_weight, random_state=21)
    
    # Si addestra il modello KNeighborsClassifier sul train set
    clf.fit(X_train, y_train)
    
    # Si salvano le predizioni effettuate
    y_pred = clf.predict(X_test)
    
    # Si salva l'accuratezza bilanciata del modello
    balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
    
    # Si salvano i vari risultati d'interesse
    result = {
        "techniques_list": technique, 
        "technique": ', '.join(technique) if technique else 'no techniques',
        "accuracy": balanced_accuracy,
        "kernel": kernel,
        "C": c,
        "gamma": gamma,
        "class_weight": class_weight
    }
    
    # Si aggiungono ai risultati finali
    result_svc.append(result)
    
    # Si definiscono le matrici binarie (1 vs All)
    mcm = multilabel_confusion_matrix(y_test, y_pred)
    
    # Si salvano i nomi delle classi
    class_names = clf.classes_
    
    # Si crea una figura con 3 righe e 3 colonne di subplot
    fig, axes = plt.subplots(3, 3, figsize=(15, 15))
    
    # Si inserisce il titolo personalizzato
    if gamma == "None":
        fig.suptitle(f"Correlation Matrix SVC\nTechniques: {', '.join(technique) if technique else 'no techniques'}\nKernel: {kernel}\nC: {c}\nClass weight: {class_weight}\nAccuracy: {balanced_accuracy}")
    else:
        fig.suptitle(f"Correlation Matrix SVC\nTechniques: {', '.join(technique) if technique else 'no techniques'}\nKernel: {kernel}\nC: {c}\nGamma: {gamma}\nClass weight: {class_weight}\nAccuracy: {balanced_accuracy}")
    
    # Si trasforma la matrice 3x3 in un array 1D
    axes = axes.flatten()

    # Si itera per ogni coppia matrice - classe corrispondente
    for i, (matrix, class_name) in enumerate(zip(mcm, class_names)):
        
        # Si salvano i valori True Negative (tn) False Positive (fp)
        # False Negative (fn) e True Positive (tp)
        tn, fp = matrix[0]
        fn, tp = matrix[1]

        # Si calcola l'accuratezza per ogni classe
        accuracy = (tp + tn) / (tp + tn + fp + fn)
        
        # Si mostra il seguente messaggio nella matrice di correlazione
        display_labels = [f"Non-{class_name}", f"{class_name}"]

        # Si visualizza la matrice di ogni classe con ConfusionMatrixDisplay
        disp = ConfusionMatrixDisplay(confusion_matrix=matrix, display_labels=display_labels)
        disp.plot(ax=axes[i], cmap="Reds")
        axes[i].set_title(f"{class_name} vs Resto\nAccuracy: {accuracy}")

    # Si visualizza la matrice di confusione generale normalizzata 
    # si vedono le percentuali di corretta previsione e dove il modello fa confusione
    cm = confusion_matrix(y_test, y_pred, normalize="true")
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
    disp.plot(ax=axes[7], cmap="Reds", values_format=".2f")
    axes[7].set_title(f"Confusion Matrix Normalized")
    axes[7].set_xticklabels(class_names, rotation=90)
    
    # Non si visualizza l'ultimo subplot disponibile
    axes[8].set_visible(False)
    
    # Si salva l'immagine nell'apposita cartella
    plt.figure(fig.number)
    plt.tight_layout()
    plt.savefig(f"{dir}/{'_'.join(technique) if technique else 'no_techniques'}_confusion_matrix")
    plt.close(fig)
    
# Si converte results in un pandas DataFrame e si ordinano i risultati
result_svc = pd.DataFrame(result_svc)
print(result_svc.sort_values("accuracy", ascending=False))

# Si ricava la migliore accuracy e si stamapa a video
print("\nMiglior configurazione:")
best_idx = result_svc['accuracy'].idxmax()
best_config_svc = result_svc.iloc[[best_idx]]
print(best_config_svc[["technique", "accuracy", "kernel", "C", "gamma", "class_weight"]])

```
Miglior configurazione:
    technique  accuracy kernel     C  gamma class_weight
normalization  0.941211    rbf  10.0  scale         None
```

In [ ]:
# Si salva la miglior configurazione trovata
best_configs_list.append(("SVC", best_config_svc))

### Random Forest

Il Random Forest è un algoritmo di machine learning basato su una *collezione di alberi decisionali indipendenti*. Ogni albero viene costruito su un *sottoinsieme casuale dei dati* (bootstrap) e delle feature, creando una "foresta" di alberi che votano collettivamente per la predizione finale.

I Random Forest offrono numerosi vantaggi: robustezza all'overfitting, capacità di gestire dataset complessi, alta accuratezza e capacità di stimare l'importanza delle feature. Possono elaborare simultaneamente feature numeriche e categoriche, tollerano valori mancanti e sono meno sensibili a outlier rispetto ai singoli alberi decisionali.

La libreria *scikit-learn* fornisce la classe [*RandomForestClassifier*](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) per implementare l'algoritmo. Di default utilizza l'indice di Gini per misurare la qualità degli split e costruisce 100 alberi nella foresta.

Si effettua un tuning degli iperparametri per trovare la migliore combinzaione di quest'ultimi (si utilizza un subset del dataset originale per ridurre la complessità).

Si noti che *RandomForest* è un algoritmo costoso computazionalmente, di conseguenza di usa un subset del dataset originale (30%). 

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# skf = StratifiedKFold è già stato definito precedentemente

# Si definisce una lista per contenere i risultati
results_randomForest = []

# Si definisce il classificatore RandomForestClassifier su cui effettuare
# il tuning degli iperparametri. 
clf = RandomForestClassifier(random_state=21, n_jobs=-1)

# Si defisce una lista del numero di alberi da testare
n_estimators = [100, 200, 300]

# Si definisce il criterio per misurare la qualità degli split
criterion = ['gini', 'entropy']

# Si definisce una lista di profondità di alberi da testare
max_depth = [5, 10, 15]

# Si definisce il parametro per la gestione dei pesi
class_weight = ['balanced', 'balanced_subsample']

# Si definisce un subset del dataset originale
X_subset, _, y_subset, _ = train_test_split(X, y, train_size=0.3, random_state=21, stratify=y)

# Si divide il subset del dataset originale in train e test con stratificazione 
# ovvero le occorrenze delle classi sono automaticamente bilanciate
# tra train e test (random state garantisce la riproducibilità)
X_train, _, y_train, _ = train_test_split(X_subset, y_subset, test_size=0.2, random_state=21, stratify=y_subset)

# Si inizializzano le due variabili utilizzate successivamente per salvare il miglior
# classificatore ttrovato tramite la grid search e cross validation
best_randomForest_score = -1
best_randomForest_clf = None

# Si itera per tutte le combinazioni di tecniche di pre processing possibili 
# e precedentemente definite
for technique in techniques:
    
    # Si definisce una lista per contenere i vari step per la cross validation
    steps = []
    
    # Se la tecnica specificata tra virgolette è presente nella lista
    # technique si aggiunge tra gli step della cross validation
    if "standardization" in technique:
        steps.append(("scaler", scaler))
        
        # Si salva il numero corretto di cluster per la standardizzazione 
        n_clusters = n_clusters_std
        
    elif "normalization" in technique:
        steps.append(("normalizer", normalizer))
        
        # Si salva il numero corretto di cluster per la normalizzazione
        n_clusters = n_clusters_std
        
    if "agglomeration" in technique:
        
        # Si definisce lo step dell'agglomerazione di feature con gli stessi 
        # parametri usati precedentemente 
        steps.append(("agglomeration", FeatureAgglomeration(
            n_clusters=n_clusters,
            metric="euclidean",
            linkage="ward"
        )))
        
    if "undersampling" in technique:
        steps.append(("undersampler", cc))
        
    elif "oversampling" in technique:
        steps.append(("oversampler", smote))
    
    # Si aggiunge il classificatore RandomForest come ultimo step
    # della cross validation
    steps.append(("classifier", clf))
    
    # Si crea la pipeline di step 
    # si utilizza ImbPipeline invece di Pipeline in quanto accetta 
    # le tecniche di sampling (oversampling/undersampling)
    pipeline = ImbPipeline(steps)
    
    # Si definisce il parametro di GridSearchCV per la ricerca degli
    # iperparametri migliori
    param_grid = {
        "classifier__n_estimators": n_estimators,
        "classifier__criterion": criterion,
        "classifier__max_depth": max_depth,
        "classifier__class_weight": class_weight
    }
    
    # Si definisce GridSearchCV, la pipeline da seguire, i parametri,
    # la tecnica per la cross validation (StratifiedKFold), lo scoring da 
    # utilizzare (balanced_accuracy in quanto il dataset è sbilanciato) e il
    # numero di core da utilizzare contemporaneamente
    grid_search = GridSearchCV(
        estimator=pipeline,
        param_grid=param_grid,
        cv=skf,
        scoring="balanced_accuracy",
        n_jobs=-1,
        verbose=1
    )
    
    # Si esegue la cross validation sul train set, per ogni tecnica si 
    # verifica quale combinazione di iperparametri ottiene l'accuratezza maggiore
    grid_search.fit(X_train, y_train)
    
    # Si verifica il risultato del modello, se migliore del precedente si salva il corrente
    if grid_search.best_score_ > best_randomForest_score:
        best_randomForest_score = grid_search.best_score_
        best_randomForest_clf = grid_search.best_estimator_
        
    # Si estrae solo il miglior risultato per questa tecnica
    best_params = grid_search.best_params_
        
    # Si definiscono i risultati per ogni tecnica, profondità e criterio
    result = {
        "techniques_list": technique,                           # Si mantiene la lista di tecniche utilizzate per comodità
        "techniques": ', '.join(technique),                     # Si converte la lista di tecniche in stringa per migliore visualizzazione
        "n_estimators": best_params["classifier__n_estimators"],     # Si salva il valore degli alberi utilizzati
        "criterion": best_params["classifier__criterion"],           # Si salva il parametro criterion
        "max_depth": best_params["classifier__max_depth"],           # Si salva la profondità
        "class_weight": best_params["classifier__class_weight"],     # Si salva il peso utilizzato
        "mean_test_score": grid_search.best_score_,                     # Si salva il valore medio dell'accuratezza bilanciata ottenuto dalle 5 iterazioni della cross validation
        "std_test_score": grid_search.cv_results_["std_test_score"][    # Si salva la deviazione standard del miglior punteggio
            grid_search.cv_results_["mean_test_score"].argmax()
        ]
    }
        
    # Si aggiungono ai risultati finali
    results_randomForest.append(result)

    # Si ibera la memoria
    del grid_search
        
# Si trasforma result in un pandas DataFrame
results_randomForest = pd.DataFrame(results_randomForest)

# Si restituisce la combinazione dei risultati migliori per ogni tecnica
print("\nMigliori configurazioni per ogni tecnica:")
print(results_randomForest[["techniques", "n_estimators", "criterion", "max_depth", "class_weight", "mean_test_score", "std_test_score"]].sort_values("mean_test_score", ascending=False)) 

```
Migliori configurazioni per ogni tecnica:
                                   techniques  n_estimators criterion  max_depth        class_weight  mean_test_score  std_test_score
                                 oversampling           200      gini         15            balanced         0.938481        0.004535
                  normalization, oversampling           200   entropy         15  balanced_subsample         0.936801        0.002530
                standardization, oversampling           100   entropy         10            balanced         0.935922        0.004005
                                                        100      gini         10            balanced         0.934762        0.007746
                                normalization           300      gini         15            balanced         0.934678        0.005034
                              standardization           200   entropy         10  balanced_subsample         0.934559        0.006729
               standardization, undersampling           100      gini         10  balanced_subsample         0.929389        0.003937
                 normalization, undersampling           100   entropy          5  balanced_subsample         0.927122        0.003353
               standardization, agglomeration           100   entropy         15            balanced         0.918924        0.003384
standardization, agglomeration, undersampling           100   entropy         15  balanced_subsample         0.918894        0.005151
 standardization, agglomeration, oversampling           300   entropy         15            balanced         0.917825        0.003587
                 normalization, agglomeration           100   entropy         10            balanced         0.911775        0.005776
   normalization, agglomeration, oversampling           300   entropy         15            balanced         0.907145        0.010949
  normalization, agglomeration, undersampling           300   entropy         10            balanced         0.905533        0.007756
                                undersampling           300   entropy         10            balanced         0.902550        0.003993
```

In [ ]:

best_randomForest_clf

Si verificano i risultati del codice precedente:
- Per ogni configurazione trovata, si applicano le medesime tecniche di preprocessing e si addestra un algoritmo RandomForest con gli ipeparametri ottimali.
- Si esegue una valutazione delle performance sul test set.
- Si visualizzano le matrici di confusione per ogni configurazione.
- Si salvano i risultati della migliore configurazione. 

In [ ]:
# Si crea la directory per RANDOM FOREST se non esiste
dir = "9 - RANDOM FOREST"

path_dir = os.path.join(os.getcwd(), dir)

if not os.path.exists(path_dir):
    os.makedirs(path_dir)

# Si crea una lista per contenere i vari risultati
result_randomForest = []

# Si ridefinisce il medesimo subset per lavorare con il test set corretto
X_subset, _, y_subset, _ = train_test_split(X, y, train_size=0.3, random_state=21, stratify=y)

# Si itera lungo le righe delle configurazioni migliori 
for row in results_randomForest.itertuples():
    # Si salva la lista contenente le tecniche di pre processing 
    technique = row.techniques_list
    
    # Si divide il subset del dataset in train e test con stratificazione
    # random_state = 21 ci garantisce che per ogni iterazione otteniamo 
    # sempre le stesse separazioni e si combinano le varie tecniche di preprocessing
    X_train, X_test, y_train, y_test = combine_preprocessing(X_subset, y_subset, technique)
    
    # Si salva il valore di n_estimators
    n_estimators = row.n_estimators
    
    # Si salva il parametro criterion
    criterion = row.criterion
    
    # Si salva la profondità
    max_depth = row.max_depth
    
    # Si salva il parametro utilizzato per i pesi
    class_weight = row.class_weight
    
    # Si definisce RandomForest con gli iperparametri definiti
    clf = RandomForestClassifier(n_estimators=n_estimators, criterion=criterion, max_depth=max_depth, class_weight=class_weight, random_state=21, n_jobs=-1)
    
    # Si addestra il modello RandomForest sul train set
    clf.fit(X_train, y_train)
    
    # Si salvano le predizioni effettuate
    y_pred = clf.predict(X_test)
    
    # Si salva l'accuratezza bilanciata del modello
    balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
    
    # Si salvano i vari risultati d'interesse
    result = {
        "techniques_list": technique,
        "technique": ', '.join(technique) if technique else 'no techniques',
        "accuracy": balanced_accuracy,
        "n_estimators": n_estimators,
        "criterion": criterion,
        "max_depth": max_depth,
        "class_weight": class_weight
    }
    
    # Si aggiungono ai risultati finali
    result_randomForest.append(result)
    
    # Si definiscono le matrici binarie (1 vs All)
    mcm = multilabel_confusion_matrix(y_test, y_pred)
    
    # Si salvano i nomi delle classi
    class_names = clf.classes_
    
    # Si crea una figura con 3 righe e 3 colonne di subplot
    fig, axes = plt.subplots(3, 3, figsize=(15, 15))
    
    # Si inserisce il titolo personalizzato
    fig.suptitle(f"Correlation Matrix RandomForest\nTechniques: {', '.join(technique) if technique else 'no techniques'}\nTrees: {n_estimators}\nCriterion: {criterion}\Profondità: {max_depth}\nClass weight: {class_weight}\nAccuracy: {balanced_accuracy}")
    
    # Si trasforma la matrice 3x3 in un array 1D
    axes = axes.flatten()

    # Si itera per ogni coppia matrice - classe corrispondente
    for i, (matrix, class_name) in enumerate(zip(mcm, class_names)):
        
        # Si salvano i valori True Negative (tn) False Positive (fp)
        # False Negative (fn) e True Positive (tp)
        tn, fp = matrix[0]
        fn, tp = matrix[1]

        # Si calcola l'accuratezza per ogni classe
        accuracy = (tp + tn) / (tp + tn + fp + fn)
        
        # Si mostra il seguente messaggio nella matrice di correlazione
        display_labels = [f"Non-{class_name}", f"{class_name}"]

        # Si visualizza la matrice di ogni classe con ConfusionMatrixDisplay
        disp = ConfusionMatrixDisplay(confusion_matrix=matrix, display_labels=display_labels)
        disp.plot(ax=axes[i], cmap="Reds")
        axes[i].set_title(f"{class_name} vs Resto\nAccuracy: {accuracy}")

    # Si visualizza la matrice di confusione generale normalizzata 
    # si vedono le percentuali di corretta previsione e dove il modello fa confusione
    cm = confusion_matrix(y_test, y_pred, normalize="true")
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
    disp.plot(ax=axes[7], cmap="Reds", values_format=".2f")
    axes[7].set_title(f"Confusion Matrix Normalized")
    axes[7].set_xticklabels(class_names, rotation=90)
    
    # Non si visualizza l'ultimo subplot disponibile
    axes[8].set_visible(False)
    
    # Si salva l'immagine nell'apposita cartella
    plt.figure(fig.number)
    plt.tight_layout()
    plt.savefig(f"{dir}/{'_'.join(technique) if technique else 'no_techniques'}_confusion_matrix")
    plt.close(fig)
    
# Si converte results in un pandas DataFrame e si ordinano i risultati
result_randomForest = pd.DataFrame(result_randomForest)
print(result_randomForest.sort_values("accuracy", ascending=False))

# Si ricava la migliore accuracy e si stamapa a video
print("\nMiglior configurazione:")
best_idx = result_randomForest['accuracy'].idxmax()
best_config_randomForest = result_randomForest.iloc[[best_idx]]
print(best_config_randomForest[["technique", "accuracy", "n_estimators", "criterion", "max_depth", "class_weight"]])

```
Miglior configurazione:
                    technique  accuracy  n_estimators criterion  max_depth class_weight
standardization, oversampling  0.938518           100   entropy         10     balanced
```

In [ ]:
# Si salva la miglior configurazione trovata
best_configs_list.append(("RandomForest", best_config_randomForest))

### Voting Classifier
Il Voting Classifier è un classificatore appartenente alla categorio degli *ensemble learning methods* ovvero quei classificatori che combinano vari modelli di apprendimento per migliorare le prestazioni (come il *Random Forest*).

Il Voting classifier funziona tramite due modalità principali: 
- *Hard Voting*: in cui ogni classificatore all'interno del Voting classifier esprime un voto per una classe, la classe che ottiene il maggior numero di voti è la classe finale.
- *Soft Voting*: in cui ogni classificatore all'interno del Voting Classifier deve fornire la probabilità di appartenenza ad ogni classe, la classe con la probabilità media più alta viene scelta (metodo valido solo con i classificatori che restituiscono le probabilità per classe).

la libreria *scikit-learn* mette a disposizione la classe [*VotingClassifier*](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingClassifier.html) per implementare l'algoritmo. By default presenta il sistema di voto impostato su *hard* e i pesi impostati su *None* (ovvero tutte le classi hanno lo stesso peso).

Si definiscono due differenti *GridSearchCV* per il tuning degli iperparametri, uno dedicato al *soft voting* e uno all'*hard voting*, questo perchè il modello *SVC* non fornisce le probabilità per classe by default e il calcolo è estremamente costoso computazionalmente. Si effettua il tuning degli iperparametri sui pesi del modello, testando varie soluzioni.

In [ ]:
from sklearn.ensemble import VotingClassifier

# skf = StratifiedKFold è già stato definito precedentemente

# Si salvano gli score ottenuti dai vari modelli per usarli come pesi
best_score_hard = [best_decisionTree_score, best_knn_score, best_naiveBayes_score, best_svc_score, best_randomForest_score]
best_score_soft = [best_decisionTree_score, best_knn_score, best_naiveBayes_score, best_randomForest_score]

# Si definisce una lista per contenere i risultati (hard e soft voting)
results_votingClassifier = []

# Si creano due VotingClassifier separati
# Si crea il classificatore per hard voting (con tutti i classificatori)
hard_voting_clf = VotingClassifier(
    estimators=[
        ("decisiontree", best_decisionTree_clf),
        ("knn", best_knn_clf),
        ("naivebayes", best_naiveBayes_clf),
        ("svc", best_svc_clf),
        ("randomforest", best_randomForest_clf)
    ],
    voting='hard',
    n_jobs=-1,
    verbose=1
)

# Si crea il classificatore per soft voting (senza SVC, in quanto non ha probability = True)
soft_voting_clf = VotingClassifier(
    estimators=[
        ("decisiontree", best_decisionTree_clf),
        ("knn", best_knn_clf),
        ("naivebayes", best_naiveBayes_clf),
        ("randomforest", best_randomForest_clf)
    ],
    voting='soft',
    n_jobs=-1,
    verbose=1
)

# Si divide il dataset iniziale in train e test con stratificazione 
# ovvero le occorrenze delle classi sono automaticamente bilanciate
# tra train e test (random state garantisce la riproducibilità)
X_train, _, y_train, _ = train_test_split(X, y, test_size=0.2, random_state=21, stratify=y)

# Si impostano i parametri dei pesi per l'hard voting
hard_param_grid = {
    "weights": [
        None,
        best_score_hard,
        [1, 1, 1, 1, 0],
        [1, 1, 1, 0, 1],
        [1, 1, 0, 1, 1],
        [1, 0, 1, 1, 1],
        [0, 1, 1, 1, 1],
        # Si impostano i pesi in base ai risultati ottenuti dai vari
        # classificatori, 5 per quello con l'accuratezza maggiore, 1 per
        # quello con l'accuratezza minore.
        [2, 3, 1, 5, 4]
    ]
}

# Si impostano i parametri dei pesi per soft voting
soft_param_grid = {
    "weights": [
        None,
        best_score_soft,
        [1, 1, 1, 0],
        [1, 1, 0, 1],
        [1, 0, 1, 1],
        [0, 1, 1, 1],
        # Si impostano i pesi in base ai risultati ottenuti dai vari
        # classificatori, 4 per quello con l'accuratezza maggiore, 1 per
        # quello con l'accuratezza minore.
        [2, 3, 1, 4]
    ]
}

# Si esegue la GridSearchCV con hard voting
hard_grid_search = GridSearchCV(
    estimator=hard_voting_clf,
    param_grid=hard_param_grid,
    cv=skf,
    scoring="balanced_accuracy",
    n_jobs=-1,
    verbose=1
)

# Si esegue la GridSearchCV con soft voting
soft_grid_search = GridSearchCV(
    estimator=soft_voting_clf,
    param_grid=soft_param_grid,
    cv=skf,
    scoring="balanced_accuracy",
    n_jobs=-1,
    verbose=1
)

# Si esegue la cross validation sul train set per entrambi i classificatori
hard_grid_search.fit(X_train, y_train)
soft_grid_search.fit(X_train, y_train)

# Si estraggono i modelli migliori
best_hardVoting_clf = hard_grid_search.best_estimator_
best_softVoting_clf = soft_grid_search.best_estimator_

# Si estraggono dei risultati per hard voting
for params, mean_test_score, std_test_score in zip(
    hard_grid_search.cv_results_["params"],
    hard_grid_search.cv_results_["mean_test_score"],
    hard_grid_search.cv_results_["std_test_score"]
):
    # Si definiscono i risultati per ogni combinazione
    result = {
        "voting": "hard",                  # Si mantiene la tipologia di voting
        "weights": params["weights"],           # Si salva la lista dei pesi utilizzata
        "mean_test_score": mean_test_score,     # Si salva il valore dell'accuratezza bilanciata media
        "std_test_score": std_test_score        # Si salva il valore della deviazione standard
    }
    
    # Si aggiungono i risultati
    results_votingClassifier.append(result)

# Si libera la memoria
del hard_grid_search

# Si estraggono i risultati per soft voting
for params, mean_test_score, std_test_score in zip(
    soft_grid_search.cv_results_["params"],
    soft_grid_search.cv_results_["mean_test_score"],
    soft_grid_search.cv_results_["std_test_score"]
):
    # Si definiscono i risultati per ogni combinazione
    result = {
        "voting": "soft",                  # Si mantiene la tipologia di voting
        "weights": params["weights"],           # Si salva la lista dei pesi utilizzata
        "mean_test_score": mean_test_score,     # Si salva il valore dell'accuratezza bilanciata medi
        "std_test_score": std_test_score        # Si salva il valore della deviazione standard
    }
    
    # Si aggiungono i risultati
    results_votingClassifier.append(result)

# Si libera la memoria
del soft_grid_search

# Si trasformano i risultati un pandas DataFrame
results_votingClassifier = pd.DataFrame(results_votingClassifier)

# Si stampano i risultati in ordine di accuratezza
print("\nMigliori configurazioni:")
print(results_votingClassifier.sort_values("mean_test_score", ascending=False))

```
Migliori configurazioni:
voting            weights  mean_test_score  std_test_score
  hard    [2, 3, 1, 5, 4]         0.937521        0.003991
  hard  [best_score_hard]         0.936997        0.004204
  hard               None         0.936980        0.004322
  hard    [1, 1, 0, 1, 1]         0.936798        0.004236
  hard    [0, 1, 1, 1, 1]         0.936356        0.002167
  soft       [2, 3, 1, 4]         0.936307        0.003814
  soft       [1, 1, 0, 1]         0.935234        0.005741
  hard    [1, 1, 1, 0, 1]         0.934846        0.003369
  hard    [1, 1, 1, 1, 0]         0.934699        0.002747
  hard    [1, 0, 1, 1, 1]         0.933753        0.004116
  soft  [best_score_soft]         0.932765        0.002429
  soft               None         0.932373        0.002199
  soft       [0, 1, 1, 1]         0.930848        0.002818
  soft       [1, 1, 1, 0]         0.929316        0.002360
  soft       [1, 0, 1, 1]         0.928375        0.002083
``` 

In [ ]:
best_hardVoting_clf

In [ ]:
best_softVoting_clf

In [ ]:
# Crea directory per VOTING CLASSIFIER se non esiste 
dir = "10 - VOTING CLASSIFIER"

path_dir = os.path.join(os.getcwd(), dir)

if not os.path.exists(path_dir):
    os.makedirs(path_dir)
    
# Si crea una lista per conservare i risultati
result_votingClassifier = []

# Si itera lungo le righe delle configurazioni migliori 
for i, row in enumerate(results_votingClassifier.itertuples()):
    
    # Si salva la tipologia di voting
    voting = row.voting
    
    # Si salvano i pesi utilizzati
    weights = row.weights
    
    # Si divide il dataset iniziale in train e test con stratificazione 
    # ovvero le occorrenze delle classi sono automaticamente bilanciate
    # tra train e test (random state garantisce la riproducibilità)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify=y)

    if voting == "soft":
        clf = VotingClassifier(
            estimators=[
                ("decisiontree", best_decisionTree_clf),
                ("knn", best_knn_clf),
                ("naivebayes", best_naiveBayes_clf),
                ("randomforest", best_randomForest_clf)
            ],
            voting=voting,
            weights=weights,
            n_jobs=-1,
            verbose=1
        )
    elif voting == "hard":
        clf = VotingClassifier(
            estimators=[
                ("decisiontree", best_decisionTree_clf),
                ("knn", best_knn_clf),
                ("naivebayes", best_naiveBayes_clf),
                ("svc", best_svc_clf),
                ("randomforest", best_randomForest_clf)
            ],
            voting=voting,
            weights=weights,
            n_jobs=-1,
            verbose=1
        )    
    
    # Si addestra il modello  
    clf.fit(X_train, y_train)
    
    # Si salvano le predizioni effettuate
    y_pred = clf.predict(X_test)
    
    # Si salva l'accuratezza bilanciata del modello
    balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
    
    # Si salvano i valori di interesse
    result = {
        "accuracy": balanced_accuracy,
        "voting": voting,
        "weights": weights
    }
    
    # Si aggiungono ai risultati
    result_votingClassifier.append(result)
    
    # Si definiscono le matrici binarie (1 vs All)
    mcm = multilabel_confusion_matrix(y_test, y_pred)
    
    # Si salvano i nomi delle classi
    class_names = clf.classes_
    
    # Si crea una figura con 3 righe e 3 colonne di subplot
    fig, axes = plt.subplots(3, 3, figsize=(15, 15))
    
    # Si inserisce il titolo personalizzato
    fig.suptitle(f"Correlation Matrix VotingClassifier\nVoting: {voting}\nWeights: {weights}")
    
    # Si trasforma la matrice 3x3 in un array 1D
    axes = axes.flatten()
    
    # Si itera per ogni coppia matrice - classe corrispondente
    for j, (matrix, class_name) in enumerate(zip(mcm, class_names)):
        
        # Si salvano i valori True Negative (tn) False Positive (fp)
        # False Negative (fn) e True Positive (tp)
        tn, fp = matrix[0]
        fn, tp = matrix[1]

        # Si calcola l'accuratezza per ogni classe
        accuracy = (tp + tn) / (tp + tn + fp + fn)
        
        # Si mostra il seguente messaggio nella matrice di correlazione
        display_labels = [f"Non-{class_name}", f"{class_name}"]

        # Si visualizza la matrice di ogni classe con ConfusionMatrixDisplay
        disp = ConfusionMatrixDisplay(confusion_matrix=matrix, display_labels=display_labels)
        disp.plot(ax=axes[j], cmap="Reds")
        axes[j].set_title(f"{class_name} vs Resto\nAccuracy: {accuracy}")

    # Si visualizza la matrice di confusione generale normalizzata 
    # si vedono le percentuali di corretta previsione e dove il modello fa confusione
    cm = confusion_matrix(y_test, y_pred, normalize="true")
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
    disp.plot(ax=axes[7], cmap="Reds", values_format=".2f")
    axes[7].set_title(f"Confusion Matrix Normalized")
    axes[7].set_xticklabels(class_names, rotation=90)
    
    # Non si visualizza l'ultimo subplot disponibile
    axes[8].set_visible(False)
    
    # Si salva l'immagine nell'apposita cartella
    plt.figure(fig.number)
    plt.tight_layout()
    plt.savefig(f"{dir}/{i}_confusion_matrix")
    plt.close(fig)

# Si converte results in un pandas DataFrame e si ordinano i risultati
result_votingClassifier = pd.DataFrame(result_votingClassifier)
print(result_votingClassifier.sort_values("accuracy", ascending=False))

# Si ricava la migliore accuracy e si stamapa a video
print("\nMiglior configurazione:")
best_idx = result_votingClassifier['accuracy'].idxmax()
best_config_votingClassifier = result_votingClassifier.iloc[[best_idx]]
print(best_config_votingClassifier)

```
Miglior configurazione:
accuracy voting          weights
0.941895   hard  [1, 1, 0, 1, 1]
```

In [ ]:
# Si salva la miglior configurazione trovata
best_configs_list.append(("VotingClassifier", best_config_votingClassifier))

In [ ]:
prova = [1, 2, 3, 4, 5, 6]
print(len(prova))

## Conclusioni

Per arrivare alla **conclusione** sono stati effettuati vari passaggi. Primo fra tutti il **tuning degli iperparametri** per gli algoritmi di classificazione *DecisionTree, K-Nearest Neighbors, Naive Bayes (Gaussian), Support Vector Classifier, Random Forest e Voting Classifier* sulle varie combinazioni di tecniche di **pre-processing**:
```txt
- Nessuna,
- standardization,
- normalization,
- undersampling,
- oversampling,
- standardization + undersampling,
- standardization + oversampling,
- normalization + undersampling,
- normalization + oversampling,
- standardization + agglomeration,
- normalization + agglomeration,
- standardization + agglomeration + undersampling,
- standardization + agglomeration + oversampling,
- normalization + agglomeration + undersampling,
- normalization + agglomeration + oversampling.
```
e su varie combinazioni di **iperparametri**:
```txt
- Decision Tree:
    -> max_depth: profondità dell'albero di classificazione;
    -> criterion: misura di qualità degli split.
- K-Nearest Neighbors: 
    -> n_neighbors: numero di istanze più prossime per la classificazione;
    -> weights: modalità di ponderazione delle istanze più prossime;
    -> metric: funzione di distanza utilizzata per misurare la prossimità tra punti nello spazio.
- Naive Bayes (Gaussian):
    -> var_smoothing: varianza massima delle caratteristiche aggiunta alle varianze per garantire stabilità.
- Support Vector Classifier:
    -> kernel: funzione per mappare i dati separandoli linearmente;
    -> C: penalità dell'errore di classificazione;
    -> gamma: influenza di ciascun punto di training;
    -> class_weight: bilancia l'importanza delle classi.
- Random Forest:
    -> n_estimator: numero di alberi decisionali;
    -> criterion: misura di qualità degli split;
    -> max_depth: profondità massima degli alberi;
    -> class_weight: bilancia l'importanza delle classi.
- Voting Classifier:
    -> voting: strategia di aggregazione delle predizioni;
    -> weights: bilancia l'importanza delle classi.
```
Il **tuning degli iperparametri** è stato eseguito sul medesimo **Train set** tramite cross validation (*tecnica di valutazione che suddivide iterativamente il dataset in sottoinsiemi complementari per l'addestramento e la validazione*).

Successivamente è stata effettuata la valutazione sul medesimo **Test set**, salvando i migliori risultati ottenuti per ogni algoritmo e configurazione.

L'intero processo ha lo scopo di determinare quale sia la migliore ottimizzazione algoritmica (come combinazione di tecniche di pre-elaborazione e configurazioni parametriche) per l'implementazione del più accurato modello di Machine Learning per il problema di classificazione multiclasse del dataset **dry_bean**.

In [ ]:
# Si riordinano i modelli in ordine di accuracy
best_configs_list = sorted(best_configs_list, key=lambda x: x[1]['accuracy'].item(), reverse=True)

# Si itera lungo best model per una stampa personalizzata
for i, (model_name, model_info) in enumerate(best_configs_list):
    
    # Si estrapola l'accuracy
    accuracy = model_info['accuracy'].item()
    
    # Se stiamo lavorando sul modello migliore
    if i == 0:
        print(f"Il miglior modello ottenuto è {model_name} con accuracy: {accuracy:.4f}")
    else:
        print(f"\nSeguito da {model_name} con accuracy: {accuracy:.4f}")
    
    # Si stampa la tecnica utilizzata (se presente)
    if 'technique' in model_info.columns:
        technique = model_info['technique'].item()
        print(f"Tecniche di pre-processing utilizzate: {technique}")
    
    print("Iperparametri:")
    # Si stampano informazioni specifiche per ciascun tipo di modello
    if model_name == 'DecisionTree':
        print(f"   {'Max Depth:':<15} {model_info['max_depth'].item()}")
        print(f"   {'Criterion:':<15} {model_info['criterion'].item()}")
    
    elif model_name == 'KNN':
        print(f"   {'Neighbors:':<15} {model_info['k_neighbors'].item()}")
        print(f"   {'Weights:':<15} {model_info['weights'].item()}")
        print(f"   {'Metric:':<15} {model_info['metric'].item()}")
    
    elif model_name == 'NaiveBayes':
        print(f"   {'Var Smoothing:':<15} {model_info['var_smoothing'].item()}")
        print(f"   {'Priors:':<15} {'Weights' if model_info['priors'].item() is not None else 'None'}")
    
    elif model_name == 'SVC':
        print(f"   {'Kernel:':<15} {model_info['kernel'].item()}")
        print(f"   {'C:':<15} {model_info['C'].item()}")
        print(f"   {'Gamma:':<15} {model_info['gamma'].item()}")
        print(f"   {'Class Weight:':<15} {model_info['class_weight'].item()}")
    
    elif model_name == 'RandomForest':
        print(f"   {'N Estimators:':<15} {model_info['n_estimators'].item()}")
        print(f"   {'Criterion:':<15} {model_info['criterion'].item()}")
        print(f"   {'Max Depth:':<15} {model_info['max_depth'].item()}")
        print(f"   {'Class Weight:':<15} {model_info['class_weight'].item()}")
    
    elif model_name == 'VotingClassifier':
        print(f"   {'Voting Type:':<15} {model_info['voting'].item()}")
        print(f"   {'Weights:':<15} {model_info['weights'].item()}")

```
Il miglior modello ottenuto è VotingClassifier con accuracy: 0.9419
Iperparametri:
   Voting Type:    hard
   Weights:        [1, 1, 0, 1, 1]

Seguito da SVC con accuracy: 0.9412
Tecniche di pre-processing utilizzate: normalization
Iperparametri:
   Kernel:         rbf
   C:              10.0
   Gamma:          scale
   Class Weight:   None

Seguito da KNN con accuracy: 0.9365
Tecniche di pre-processing utilizzate: standardization, oversampling
Iperparametri:
   Neighbors:      14
   Weights:        distance
   Metric:         euclidean

Seguito da DecisionTree con accuracy: 0.9260
Tecniche di pre-processing utilizzate: standardization, oversampling
Iperparametri:
   Max Depth:      9
   Criterion:      gini

Seguito da NaiveBayes con accuracy: 0.9100
Tecniche di pre-processing utilizzate: oversampling
Iperparametri:
   Var Smoothing:  1e-16
   Priors:         Weights

Seguito da RandomForest con accuracy: 0.8816
Tecniche di pre-processing utilizzate: normalization, agglomeration, oversampling
Iperparametri:
   N Estimators:   300
   Criterion:      entropy
   Max Depth:      15
   Class Weight:   balanced
```